### This notebook plots the perforance of equal shares and utilitarian greedy (along with their exclusive version) for the impact metrics of budget share, winning rate, cost representation and project representation in the form of a box plot. Additional boxplot for the novelty metrics along each impact area is also shown in the same plot at the end

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [ ]:
# use this code block to set column and row viewing size/width
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 30)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

pd.options.display.float_format = '{:.4f}'.format

In [ ]:
# read the metadata csv
pbsummary_df = pd.read_csv('../metadata.csv', delimiter=';')
pbsummary_df = pbsummary_df.drop_duplicates()
pbsummary_df

pbsummary_df['subunit'].fillna(value='all', inplace=True)

In [ ]:
# Get election id and vote type, so we know better to segragate approval and score votings
pbsummary_with_vote_type = pbsummary_df[['election_id', 'vote_type']]

In [ ]:
# reading the projects CSV and loading to dataframe
pbprojects_df = pd.read_csv('../projects.csv', delimiter=';')
pbprojects_df.drop_duplicates(inplace=True)
print(pbprojects_df.shape)

# merge the column vote_type into pbprojects_df
pbprojects_df = pd.merge(pbprojects_df, pbsummary_with_vote_type, on='election_id', how='inner')
print(pbprojects_df.shape)

In [ ]:
# Excluding that single project id by checking with particular election id and project id
valid_pbprojects_df = pbprojects_df[pbprojects_df['cost'] != 0]
print(pbprojects_df.shape)
print(valid_pbprojects_df.shape)

In [ ]:
# There are citywide elections (unit-level); i.e. subunit is na, for such records fill na columns with values for subunit as all
valid_pbprojects_df['subunit'].fillna(value='all', inplace=True)

In [ ]:
# setting up vote_percent column
valid_pb_projects_total_selections = valid_pbprojects_df.groupby(['election_id'])['votes'].sum().reset_index()
valid_pb_projects_total_selections.rename(columns={'votes': 'total_votes_selection'}, inplace=True)
valid_pbprojects_df = valid_pbprojects_df.merge(valid_pb_projects_total_selections, on='election_id', how='inner')
valid_pbprojects_df['vote_percent'] = round((valid_pbprojects_df['votes'] / valid_pbprojects_df['total_votes_selection'] * 100),3)

In [ ]:
# Election ID for City Idea and Green Million
aarau_election_id = valid_pbprojects_df[valid_pbprojects_df['country'] == 'Switzerland'].groupby(['election_id']).first().reset_index()['election_id']
aarau_election_id = aarau_election_id.values[0]

green_budget_election_id = valid_pbprojects_df[valid_pbprojects_df['unit'] == 'Wieliczka'].groupby(['election_id']).first().reset_index()['election_id']
green_budget_election_id = green_budget_election_id.values[0]

In [ ]:
# setting additional column for used budget with MES aggregation
print(valid_pbprojects_df.shape)

elections_with_mes_winners = valid_pbprojects_df[valid_pbprojects_df['is_mes_winner'] == True]
elections_with_greedy_winners = valid_pbprojects_df[valid_pbprojects_df['is_greedy_winner'] == True]


# Getting the total budget usage for MES winning projects of each election_id (grouped) and adding a new column to denote that value
valid_pb_projects_mes_budget_usage = valid_pbprojects_df[valid_pbprojects_df['is_mes_winner'] == True].groupby(['election_id'])['cost'].sum().reset_index()
valid_pb_projects_mes_budget_usage.rename(columns={'cost': 'mes_total_budget_usage'}, inplace=True)
valid_pbprojects_df = valid_pbprojects_df.merge(valid_pb_projects_mes_budget_usage, on='election_id', how='inner')
valid_pbprojects_df['mes_budget_usage_percent'] = round((valid_pbprojects_df['mes_total_budget_usage'] / valid_pbprojects_df['total_budget'] * 100),3)

## Fill empty values for category with the label 'uncategorized' to aid in further data analysis

In [ ]:
valid_pbprojects_df['category'].value_counts()

# Checking to see if there are empty values for category in the entire project dataset
na_category_count = valid_pbprojects_df['category'].isna().sum()
print("Empty category values for PB projects are: ", na_category_count)

# Fill such empty values of category with the label uncategorized, so that it can aid in further data preprocessing
valid_pbprojects_df['category'].fillna('uncategorized', inplace=True)


print("Emtpy category values after filling na: ", valid_pbprojects_df['category'].isna().sum())
print("`uncategorized` category count for valid pb projects: ", valid_pbprojects_df[valid_pbprojects_df['category'] == 'uncategorized'].shape[0])

#### The column `category` contains multiple categories embedded into the column separated by commas, let's do some data processing so that we select only first signifiying category. We store this into a new column called `major_category`

In [ ]:
def trimCategoryText(category_text):
    return category_text.split(',')[0]

valid_pbprojects_df['major_category'] = valid_pbprojects_df['category'].apply(trimCategoryText)
valid_pbprojects_df['major_category'].nunique()


## Create additional columns for each category label

In [ ]:
# Split the Categories column and create a set of unique categories
categories_set = set(category.strip() for categories in valid_pbprojects_df['category'] for category in categories.split(','))

# Create new columns with default value 0
for category in categories_set:
    valid_pbprojects_df[f'category_{category}'] = 0

# Iterate through rows and update the new columns
for index, row in valid_pbprojects_df.iterrows():
    categories = row['category'].split(',')
    for category in categories:
        valid_pbprojects_df.at[index, f'category_{category.strip()}'] = 1
    
    if(len(categories) == 1 and (categories[0] == 'uncategorized')):
        valid_pbprojects_df.at[index, 'category_labels_count'] = 0
    else:
        valid_pbprojects_df.at[index, 'category_labels_count'] = int(len(categories))

## Check to see if there are other labelled categories there were in between the commas but not in the major categories. This is not at a single project level, but the entire dataset level

In [ ]:
major_categories_list = valid_pbprojects_df['major_category'].unique().tolist()
print(major_categories_list)
unlisted_minor_categories = []

for idx, row in valid_pbprojects_df.iterrows():
    category_text = row['category']
    cat_list = category_text.split(',')
    for item in cat_list:
        if item not in major_categories_list:
            unlisted_minor_categories.append(item)


print("Unlisted minor categories: ", unlisted_minor_categories)


## For each instance of PB normalize project costs with min-max normalization

In [ ]:
instance_grouped_max_pbprojects_df = valid_pbprojects_df.groupby(['election_id'])['cost'].max().reset_index()
instance_grouped_max_pbprojects_df.rename(columns={'cost': 'election_project_max_cost'}, inplace=True)

instance_grouped_min_pbprojects_df = valid_pbprojects_df.groupby(['election_id'])['cost'].min().reset_index()
instance_grouped_min_pbprojects_df.rename(columns={'cost': 'election_project_min_cost'}, inplace=True)

instance_grouped_total_projects_cost_df = valid_pbprojects_df.groupby(['election_id'])['cost'].sum().reset_index()
instance_grouped_total_projects_cost_df.rename(columns={'cost': 'total_projects_cost'}, inplace=True)

print("Valid PB Projects Shape before merge: ", valid_pbprojects_df.shape)
valid_pbprojects_df = valid_pbprojects_df.merge(instance_grouped_max_pbprojects_df, on='election_id', how='inner')
valid_pbprojects_df = valid_pbprojects_df.merge(instance_grouped_min_pbprojects_df, on='election_id', how='inner')
valid_pbprojects_df = valid_pbprojects_df.merge(instance_grouped_total_projects_cost_df, on='election_id', how='inner')
print("After merge: ", valid_pbprojects_df.shape)

def applyMinMaxNormalization(row):
    if ((row['cost'] == row['election_project_max_cost']) & (row['cost'] == row['election_project_min_cost'])):
        return 0.5
    else:
        return ((row['cost'] - row['election_project_min_cost']) / (row['election_project_max_cost'] - row['election_project_min_cost']))

valid_pbprojects_df['normalized_cost'] = valid_pbprojects_df.apply(lambda row: applyMinMaxNormalization(row), axis=1)

In [ ]:
# Group valid_pbprojects_df and create a new dataframe for each grouped average results
instance_overall_avg_cost = valid_pbprojects_df.groupby(['election_id'])['normalized_cost'].mean().reset_index()
instance_overall_avg_cost.rename(columns={'normalized_cost': 'normalized_overall_avg_cost'}, inplace=True)

instance_greedy_winner_avg_cost = valid_pbprojects_df[(valid_pbprojects_df['is_greedy_winner'] == True)].groupby('election_id')['normalized_cost'].mean().reset_index()
instance_greedy_winner_avg_cost.rename(columns={'normalized_cost': 'normalized_greedy_avg_cost'}, inplace=True)
instance_overall_df = instance_overall_avg_cost.merge(instance_greedy_winner_avg_cost, on='election_id', how='inner')


instance_mes_winner_avg_cost = valid_pbprojects_df[(valid_pbprojects_df['is_mes_winner'] == True)].groupby('election_id')['normalized_cost'].mean().reset_index()
instance_mes_winner_avg_cost.rename(columns={'normalized_cost': 'normalized_mes_avg_cost'}, inplace=True)
instance_overall_df = instance_overall_df.merge(instance_mes_winner_avg_cost, on='election_id', how='inner')

instance_either_winner_avg_cost = valid_pbprojects_df[(valid_pbprojects_df['is_mes_winner'] ==True) | (valid_pbprojects_df['is_greedy_winner'] == True)].groupby('election_id')['normalized_cost'].mean().reset_index()
instance_either_winner_avg_cost.rename(columns={'normalized_cost': 'normalized_either_avg_cost'}, inplace=True)
instance_overall_df = instance_overall_df.merge(instance_either_winner_avg_cost, on='election_id', how='inner')

instance_neither_winner_avg_cost = valid_pbprojects_df[(valid_pbprojects_df['is_mes_winner'] == False) & (valid_pbprojects_df['is_greedy_winner'] == False)].groupby('election_id')['normalized_cost'].mean().reset_index()
instance_neither_winner_avg_cost.rename(columns={'normalized_cost': 'normalized_neither_avg_cost'}, inplace=True)
instance_overall_df = instance_overall_df.merge(instance_neither_winner_avg_cost, on='election_id', how='left')

instance_both_winner_avg_cost = valid_pbprojects_df[(valid_pbprojects_df['is_mes_winner'] == True) & (valid_pbprojects_df['is_greedy_winner'] == True)].groupby('election_id')['normalized_cost'].mean().reset_index()
instance_both_winner_avg_cost.rename(columns={'normalized_cost': 'normalized_both_avg_cost'}, inplace=True)
instance_overall_df = instance_overall_df.merge(instance_both_winner_avg_cost, on='election_id', how='left')

instance_only_greedy_winner_avg_cost = valid_pbprojects_df[(valid_pbprojects_df['is_mes_winner'] == False) & (valid_pbprojects_df['is_greedy_winner'] == True)].groupby('election_id')['normalized_cost'].mean().reset_index()
instance_only_greedy_winner_avg_cost.rename(columns={'normalized_cost': 'normalized_only_greedy_avg_cost'}, inplace=True)
instance_overall_df = instance_overall_df.merge(instance_only_greedy_winner_avg_cost, on='election_id', how='left')

instance_only_mes_winner_avg_cost = valid_pbprojects_df[(valid_pbprojects_df['is_mes_winner'] == True) & (valid_pbprojects_df['is_greedy_winner'] == False)].groupby('election_id')['normalized_cost'].mean().reset_index()
instance_only_mes_winner_avg_cost.rename(columns={'normalized_cost': 'normalized_only_mes_avg_cost'}, inplace=True)
instance_overall_df = instance_overall_df.merge(instance_only_mes_winner_avg_cost, on='election_id', how='left')

# Fill non-existent values for instance_overall_df with values equal to -1. -1 because we had already normalized values between 0 and 1
instance_overall_df.fillna(-1, inplace=True)
print(instance_overall_df.shape)

In [ ]:
valid_pbprojects_df.columns

## Create a replica of the valid_pbprojects_df and find the distribution % of PB winners across different categories using MES against Greedy

In [ ]:
categorization_df = valid_pbprojects_df
categorization_df = categorization_df[[
       'election_id', 'unit', 'subunit', 'instance', 'project_id', 'cost', 'target', 'total_projects_cost', 'votes', 'score', 'is_mes_winner', 'is_greedy_winner', 'is_phragmen_winner', 'category', 'category_public transit and roads',
       'category_health', 'category_welfare', 'category_uncategorized',
       'category_public space', 'category_urban greenery', 'category_culture',
       'category_education', 'category_sport',
       'category_environmental protection', 'category_labels_count'
]]
categorization_df.rename(columns={'category_education': 'education', 'category_public transit and roads': 'public_transit_and_roads', 'category_health': 'health', 'category_welfare': 'welfare', 'category_uncategorized':'uncategorized', 'category_public space': 'public_space', 'category_urban greenery': 'urban_greenery', 'category_culture': 'culture', 'category_sport': 'sport', 'category_environmental protection': 'env_protection', 'category_labels_count': 'total_tags' }, inplace=True)


# In my earlier logic, I had put uncategorized to each undefined project categorization, however, this did not increase the total category counts. 
# So using this, we can the filter to remove uncategorized pb instances where uncategorized value is gre
# ater than 0
# Also, accordingly, uncategorized > 0 and total count values > 0 must not exists; sanity check
empty_df = categorization_df[(categorization_df['uncategorized'] > 0) & (categorization_df['total_tags'] > 0)]
print("Size of returned df must be 0: ", empty_df.shape)

# Apply an additional filter to remove uncategorized values from the categorization_df
print("Before applying filter shape of categorization DF is: ", categorization_df.shape)
categorization_df = categorization_df[categorization_df['uncategorized'] == 0]
print("After filtering, shape of categorization_df is: ", categorization_df.shape)
## The output shape of 10830 x 22, matches with valid_pbprojects_df rows having major_category values other than `uncategorized`
# So our data filtering is correct



# Taking the sum of each category after having goruped by election id for mes_winners and rename the columns
mes_winners_grouped_categorization_df = categorization_df[categorization_df['is_mes_winner'] == True].groupby(['election_id'])[['public_transit_and_roads', 'health', 'welfare', 'uncategorized', 'public_space', 'urban_greenery', 'culture', 'education', 'sport', 'env_protection', 'total_tags']].sum()
only_mes_winners_grouped_categorization_df = categorization_df[(categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == False)].groupby(['election_id'])[['public_transit_and_roads', 'health', 'welfare', 'uncategorized', 'public_space', 'urban_greenery', 'culture', 'education', 'sport', 'env_protection', 'total_tags']].sum()

greedy_winners_grouped_categorization_df = categorization_df[categorization_df['is_greedy_winner'] == True].groupby(['election_id'])[['public_transit_and_roads', 'health', 'welfare', 'uncategorized', 'public_space', 'urban_greenery', 'culture', 'education', 'sport', 'env_protection', 'total_tags']].sum()
only_greedy_winners_grouped_categorization_df = categorization_df[(categorization_df['is_greedy_winner'] == True) & (categorization_df['is_mes_winner'] == False)].groupby(['election_id'])[['public_transit_and_roads', 'health', 'welfare', 'uncategorized', 'public_space', 'urban_greenery', 'culture', 'education', 'sport', 'env_protection', 'total_tags']].sum()

# Apply filters to remove PB instances that have uncategorized datasets
# Group by election IDs and sum the values of each category. If an instance has no values of the summed values in either categories, those are to be removed as uncategorized

# Create columns to store selection of percentages
def addPercentageColumns(df):
       categories_set = ['education', 'public_transit_and_roads', 'health', 'welfare', 'uncategorized', 'public_space', 'urban_greenery', 'culture', 'sport', 'env_protection']

       # Create new columns for percentage values
       for category in categories_set:
              df[f'percentage_{category}'] = (df[category] / df['total_tags']) * 100

# Apply the addPercentageColumns for each of the above 4 dataframes
addPercentageColumns(mes_winners_grouped_categorization_df)
addPercentageColumns(only_mes_winners_grouped_categorization_df)
addPercentageColumns(greedy_winners_grouped_categorization_df)
addPercentageColumns(only_greedy_winners_grouped_categorization_df)

print(mes_winners_grouped_categorization_df.shape)

pb_instance_with_labels = valid_pbprojects_df[valid_pbprojects_df['major_category'] != 'uncategorized'].groupby('election_id').count()
print("PB instances with valid labels: ", pb_instance_with_labels.shape)
# The row value from the above shape matches with the MES winners and Greedy winners after grouping

mes_winners_grouped_categorization_df = mes_winners_grouped_categorization_df.add_prefix('mes_')
greedy_winners_grouped_categorization_df = greedy_winners_grouped_categorization_df.add_prefix('greedy_')

only_greedy_winners_grouped_categorization_df = only_greedy_winners_grouped_categorization_df.add_prefix('only_greedy_')
only_mes_winners_grouped_categorization_df = only_mes_winners_grouped_categorization_df.add_prefix('only_mes_')

winners_grouped_categorization_df = mes_winners_grouped_categorization_df.merge(greedy_winners_grouped_categorization_df, how='inner', on='election_id')

# because only greedy has one more row (election instance) than only mes, verified by the code block below
exclusive_winners_grouped_categorization_df = only_greedy_winners_grouped_categorization_df.merge(only_mes_winners_grouped_categorization_df, how='inner', on='election_id')

In [ ]:
# Concat works here to merge the winners grouped data and exclusive winners data, because they share the common index
grouped_categorization_df = pd.concat([winners_grouped_categorization_df, exclusive_winners_grouped_categorization_df], axis=1)
print(grouped_categorization_df.shape)
grouped_categorization_df

# drop uncategorized and their respective % columns from the dataset, because they are all zero, since we filtered out uncategorized data earlier
grouped_categorization_df.drop(columns=['mes_uncategorized', 'mes_percentage_uncategorized', 'greedy_uncategorized', 'greedy_percentage_uncategorized', 'only_mes_uncategorized', 'only_mes_percentage_uncategorized', 'only_greedy_uncategorized', 'only_greedy_percentage_uncategorized'], inplace=True)
print("After dropping, new shape is: ", grouped_categorization_df.shape)

In [ ]:
# some globals to be used for the code snippets below
categories_set = ['education', 'public_transit_and_roads', 'health', 'welfare', 'public_space', 'urban_greenery', 'culture', 'sport', 'env_protection']

## Create sorted ordering for the 276 PB instances with their % values of winnings in each category that have only MES winnings and only Greedy winnings

In [ ]:
grouped_categorization_df['diff_pct_education'] = grouped_categorization_df['greedy_percentage_education'] - grouped_categorization_df['mes_percentage_education'] 
grouped_categorization_df['diff_pct_health'] = grouped_categorization_df['greedy_percentage_health'] - grouped_categorization_df['mes_percentage_health']
grouped_categorization_df['diff_pct_welfare'] = grouped_categorization_df['greedy_percentage_welfare'] - grouped_categorization_df['mes_percentage_welfare']
grouped_categorization_df['diff_pct_sport'] = grouped_categorization_df['greedy_percentage_sport'] - grouped_categorization_df['mes_percentage_sport']
grouped_categorization_df['diff_pct_public_space'] = grouped_categorization_df['greedy_percentage_public_space'] - grouped_categorization_df['mes_percentage_public_space']
grouped_categorization_df['diff_pct_public_transit_and_roads'] = grouped_categorization_df['greedy_percentage_public_transit_and_roads'] - grouped_categorization_df['mes_percentage_public_transit_and_roads']
grouped_categorization_df['diff_pct_env_protection'] = grouped_categorization_df['greedy_percentage_env_protection'] - grouped_categorization_df['mes_percentage_env_protection']
grouped_categorization_df['diff_pct_culture'] = grouped_categorization_df['greedy_percentage_culture'] - grouped_categorization_df['mes_percentage_culture']
grouped_categorization_df['diff_pct_urban_greenery'] = grouped_categorization_df['greedy_percentage_urban_greenery'] - grouped_categorization_df['mes_percentage_urban_greenery']

grouped_categorization_df['only_diff_pct_education'] = grouped_categorization_df['only_greedy_percentage_education'] - grouped_categorization_df['only_mes_percentage_education'] 
grouped_categorization_df['only_diff_pct_health'] = grouped_categorization_df['only_greedy_percentage_health'] - grouped_categorization_df['only_mes_percentage_health']
grouped_categorization_df['only_diff_pct_welfare'] = grouped_categorization_df['only_greedy_percentage_welfare'] - grouped_categorization_df['only_mes_percentage_welfare']
grouped_categorization_df['only_diff_pct_sport'] = grouped_categorization_df['only_greedy_percentage_sport'] - grouped_categorization_df['only_mes_percentage_sport']
grouped_categorization_df['only_diff_pct_public_space'] = grouped_categorization_df['only_greedy_percentage_public_space'] - grouped_categorization_df['only_mes_percentage_public_space']
grouped_categorization_df['only_diff_pct_public_transit_and_roads'] = grouped_categorization_df['only_greedy_percentage_public_transit_and_roads'] - grouped_categorization_df['only_mes_percentage_public_transit_and_roads']
grouped_categorization_df['only_diff_pct_env_protection'] = grouped_categorization_df['only_greedy_percentage_env_protection'] - grouped_categorization_df['only_mes_percentage_env_protection']
grouped_categorization_df['only_diff_pct_culture'] = grouped_categorization_df['only_greedy_percentage_culture'] - grouped_categorization_df['only_mes_percentage_culture']
grouped_categorization_df['only_diff_pct_urban_greenery'] = grouped_categorization_df['only_greedy_percentage_urban_greenery'] - grouped_categorization_df['only_mes_percentage_urban_greenery']


grouped_categorization_df_na_removed = grouped_categorization_df.dropna()

# Check the output of two different values just for confirmation
# grouped_categorization_df[['mes_percentage_health', 'greedy_percentage_health', 'diff_pct_health', 'only_diff_pct_education', 'only_greedy_percentage_education', 'only_mes_percentage_education']]

## Use the categorization_df dataframe as a base and add new metrics necessary for calculating cost utilization and relative winners for each category of each PB instance

In [ ]:
# for each category; add columns that signify the total cost of each category per PB instance
# Create columns to store selection of percentages

# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[categorization_df[category] == 1].groupby(['election_id'])['cost'].sum().reset_index()
    temp_grouped_df.rename(columns={'cost': f'{category}_total_cost'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'{category}_total_cost'].fillna(0, inplace=True)

In [ ]:
# Add 4 columns that denote; MES Winners Projects Count, MES Winners Project Costs, Greedy Winners Projects Count and Greedy Winners Project Cost; (overall)
election_grouped_mes_winners_total_cost = categorization_df[(categorization_df['is_mes_winner'] == True)].groupby(['election_id'])['cost'].sum().reset_index()
election_grouped_mes_winners_total_cost.rename(columns={'cost': 'mes_winners_total_cost'}, inplace=True)

election_grouped_greedy_winners_total_cost = categorization_df[(categorization_df['is_greedy_winner'] == True)].groupby(['election_id'])['cost'].sum().reset_index()
election_grouped_greedy_winners_total_cost.rename(columns={'cost': 'greedy_winners_total_cost'}, inplace=True)

election_grouped_mes_winners_total_project_count = categorization_df[(categorization_df['is_mes_winner'] == True)].groupby(['election_id'])['project_id'].count().reset_index()
election_grouped_mes_winners_total_project_count.rename(columns={'project_id': 'mes_winners_projects_count'}, inplace=True)

election_grouped_greedy_winners_total_project_count = categorization_df[(categorization_df['is_greedy_winner'] == True)].groupby(['election_id'])['project_id'].count().reset_index()
election_grouped_greedy_winners_total_project_count.rename(columns={'project_id': 'greedy_winners_projects_count'}, inplace=True)

# Merge these dataset with categorization_df
categorization_df = categorization_df.merge(election_grouped_mes_winners_total_cost, on='election_id', how='inner')
categorization_df = categorization_df.merge(election_grouped_greedy_winners_total_cost, on='election_id', how='inner')
categorization_df = categorization_df.merge(election_grouped_mes_winners_total_project_count, on='election_id', how='inner')
categorization_df = categorization_df.merge(election_grouped_greedy_winners_total_project_count, on='election_id', how='inner')

In [ ]:
# Add another 4 columns that denote; only MES Winners Projects Count, only MES Winners Project Costs, only Greedy Winners Projects Count and only Greedy Winners Project Cost; (overall)
election_grouped_only_mes_winners_total_cost = categorization_df[(categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == False)].groupby(['election_id'])['cost'].sum().reset_index()
election_grouped_only_mes_winners_total_cost.rename(columns={'cost': 'only_mes_winners_total_cost'}, inplace=True)

election_grouped_only_greedy_winners_total_cost = categorization_df[(categorization_df['is_greedy_winner'] == True) & (categorization_df['is_mes_winner'] == False)].groupby(['election_id'])['cost'].sum().reset_index()
election_grouped_only_greedy_winners_total_cost.rename(columns={'cost': 'only_greedy_winners_total_cost'}, inplace=True)

election_grouped_only_mes_winners_total_project_count = categorization_df[(categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == False)].groupby(['election_id'])['project_id'].count().reset_index()
election_grouped_only_mes_winners_total_project_count.rename(columns={'project_id': 'only_mes_winners_projects_count'}, inplace=True)

election_grouped_only_greedy_winners_total_project_count = categorization_df[(categorization_df['is_greedy_winner'] == True) & (categorization_df['is_mes_winner'] == False)].groupby(['election_id'])['project_id'].count().reset_index()
election_grouped_only_greedy_winners_total_project_count.rename(columns={'project_id': 'only_greedy_winners_projects_count'}, inplace=True)

# Merge these dataset with categorization_df; for only, we will need to use left join, because there can be cases where
# both outcomes are the same, in such cases only values don't exist; so left join is needed
categorization_df = categorization_df.merge(election_grouped_only_mes_winners_total_cost, on='election_id', how='left')
categorization_df = categorization_df.merge(election_grouped_only_greedy_winners_total_cost, on='election_id', how='left')
categorization_df = categorization_df.merge(election_grouped_only_mes_winners_total_project_count, on='election_id', how='left')
categorization_df = categorization_df.merge(election_grouped_only_greedy_winners_total_project_count, on='election_id', how='left')

In [ ]:
# Merge categorization_df with pbsummary to get total projects in all elections
pbsummary_num_votes = pbsummary_df[['election_id', 'num_projects']]
categorization_df = categorization_df.merge(pbsummary_num_votes, on='election_id', how='inner')

In [ ]:
# Adding total cost of each category in each election for greedy winners

# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    greedy_winners_category_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_greedy_winner'] == True)].groupby(['election_id'])['cost'].sum().reset_index()
    greedy_winners_category_grouped_df.rename(columns={'cost': f'greedy_winners_{category}_total_cost'}, inplace=True)
    
    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(greedy_winners_category_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'greedy_winners_{category}_total_cost'].fillna(0, inplace=True)

# Adding total cost of each category in each election for MES winners
for category in categories_set:
    # temporarily grouped df 
    mes_winners_category_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_mes_winner'] == True)].groupby(['election_id'])['cost'].sum().reset_index()
    mes_winners_category_grouped_df.rename(columns={'cost': f'mes_winners_{category}_total_cost'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(mes_winners_category_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'mes_winners_{category}_total_cost'].fillna(0, inplace=True)


In [ ]:
# Project counts in each category; just overall, not winners in any sense

# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[categorization_df[category] == 1].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df.rename(columns={'project_id': f'{category}_projects_count'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'{category}_projects_count'].fillna(0, inplace=True)

In [ ]:
# Project counts in each category that are Greedy winners

# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_greedy_winner'] == True)].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df.rename(columns={'project_id': f'greedy_winners_{category}_projects_count'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'greedy_winners_{category}_projects_count'].fillna(0, inplace=True)


# Project counts in each category that are MES Winners
# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_mes_winner'] == True)].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df.rename(columns={'project_id': f'mes_winners_{category}_projects_count'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'mes_winners_{category}_projects_count'].fillna(0, inplace=True)

In [ ]:
# Project counts in each category that are Greedy winners only

# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_greedy_winner'] == True) & (categorization_df['is_mes_winner'] == False)].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df.rename(columns={'project_id': f'only_greedy_winners_{category}_projects_count'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'only_greedy_winners_{category}_projects_count'].fillna(0, inplace=True)


# Project counts in each category that are MES Winners only
# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == False)].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df.rename(columns={'project_id': f'only_mes_winners_{category}_projects_count'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'only_mes_winners_{category}_projects_count'].fillna(0, inplace=True)

In [ ]:
# Project counts in each category that are Greedy winners only

# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_greedy_winner'] == True) & (categorization_df['is_mes_winner'] == False)].groupby(['election_id'])['cost'].sum().reset_index()
    temp_grouped_df.rename(columns={'cost': f'only_greedy_winners_{category}_total_cost'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'only_greedy_winners_{category}_total_cost'].fillna(0, inplace=True)


# Project counts in each category that are MES Winners only
# Perform grouping based on catgories and then add to respective costs columns
for category in categories_set:
    # temporarily grouped df 
    temp_grouped_df = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == False)].groupby(['election_id'])['cost'].sum().reset_index()
    temp_grouped_df.rename(columns={'cost': f'only_mes_winners_{category}_total_cost'}, inplace=True)

    # Check if not using assignment but just using left join works or not; of course it wouldn't work, because merge returns the results in an entirely different dataset
    categorization_df = categorization_df.merge(temp_grouped_df, how='left', on='election_id')

# For the above columns, there can be NA values, replace them with zeros
for category in categories_set:
    categorization_df[f'only_mes_winners_{category}_total_cost'].fillna(0, inplace=True)

In [ ]:
## both winners total cost and both winners total count
# project cost
for category in categories_set:
    temp_grouped_df_v2 = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == True)].groupby(['election_id'])['cost'].sum().reset_index()
    temp_grouped_df_v2.rename(columns={'cost': f'both_winners_{category}_total_cost'}, inplace=True)

    categorization_df = categorization_df.merge(temp_grouped_df_v2, how='left', on='election_id')

for category in categories_set:
    categorization_df[f'both_winners_{category}_total_cost'].fillna(0, inplace=True)

# project counts
for category in categories_set:
    temp_grouped_df_v2 = categorization_df[(categorization_df[category] == 1) & (categorization_df['is_mes_winner'] == True) & (categorization_df['is_greedy_winner'] == True)].groupby(['election_id'])['project_id'].count().reset_index()
    temp_grouped_df_v2.rename(columns={'project_id': f'both_winners_{category}_projects_count'}, inplace=True)

    categorization_df = categorization_df.merge(temp_grouped_df_v2, how='left', on='election_id')

for category in categories_set:
    categorization_df[f'both_winners_{category}_projects_count'].fillna(0, inplace=True)

##  Metrics for Budget Share and Winning Rate

In [ ]:
# metrics calculation required for relative winners
print("Current shape of categorization df is: ", categorization_df.shape)

for category in categories_set:
    categorization_df[f'greedy_relative_winners_cost_pct_{category}'] = 1 * categorization_df[f'greedy_winners_{category}_total_cost'] / categorization_df['greedy_winners_total_cost']
    categorization_df[f'mes_relative_winners_cost_pct_{category}'] = 1 * categorization_df[f'mes_winners_{category}_total_cost'] / categorization_df['mes_winners_total_cost']
    categorization_df[f'diff_greedy_mes_relative_winners_cost_pct_{category}'] = categorization_df[f'greedy_relative_winners_cost_pct_{category}'] - categorization_df[f'mes_relative_winners_cost_pct_{category}']

    categorization_df[f'greedy_relative_winners_count_pct_{category}'] = 1 * categorization_df[f'greedy_winners_{category}_projects_count'] / categorization_df['greedy_winners_projects_count']
    categorization_df[f'mes_relative_winners_count_pct_{category}'] = 1 * categorization_df[f'mes_winners_{category}_projects_count'] / categorization_df['mes_winners_projects_count']
    categorization_df[f'diff_greedy_mes_relative_winners_count_pct_{category}'] = categorization_df[f'greedy_relative_winners_count_pct_{category}'] - categorization_df[f'mes_relative_winners_count_pct_{category}']
    
    categorization_df[f'only_greedy_relative_winners_cost_pct_{category}'] = 1 * categorization_df[f'only_greedy_winners_{category}_total_cost'] / categorization_df['greedy_winners_total_cost']
    categorization_df[f'only_mes_relative_winners_cost_pct_{category}'] = 1 * categorization_df[f'only_mes_winners_{category}_total_cost'] / categorization_df['mes_winners_total_cost']
    categorization_df[f'diff_go_mo_relative_winners_cost_pct_{category}'] = categorization_df[f'only_greedy_relative_winners_cost_pct_{category}'] - categorization_df[f'only_mes_relative_winners_cost_pct_{category}']
    
    categorization_df[f'only_greedy_relative_winners_count_pct_{category}'] = 1 * categorization_df[f'only_greedy_winners_{category}_projects_count'] / categorization_df[f'greedy_winners_projects_count']
    categorization_df[f'only_mes_relative_winners_count_pct_{category}'] = 1 * categorization_df[f'only_mes_winners_{category}_projects_count'] / categorization_df[f'mes_winners_projects_count']
    categorization_df[f'diff_go_mo_relative_winners_count_pct_{category}'] = categorization_df[f'only_greedy_relative_winners_count_pct_{category}'] - categorization_df[f'only_mes_relative_winners_count_pct_{category}']

print("After adding new columns, shape should increase by 108 cols: ", categorization_df.shape)

In [ ]:
# additional metrics for average; relative winners cost / number
for category in categories_set:
    categorization_df[f'greedy_relative_winners_avg_cost_{category}'] = categorization_df[f'greedy_relative_winners_cost_pct_{category}'] / categorization_df[f'greedy_relative_winners_count_pct_{category}']
    categorization_df[f'mes_relative_winners_avg_cost_{category}'] = categorization_df[f'mes_relative_winners_cost_pct_{category}'] / categorization_df[f'mes_relative_winners_count_pct_{category}']
    categorization_df[f'diff_ug_mes_relative_winners_avg_cost_{category}'] = categorization_df[f'greedy_relative_winners_avg_cost_{category}'] - categorization_df[f'mes_relative_winners_avg_cost_{category}'] 

    categorization_df[f'only_greedy_relative_winners_avg_cost_{category}'] = categorization_df[f'only_greedy_relative_winners_cost_pct_{category}'] / categorization_df[f'only_greedy_relative_winners_count_pct_{category}']
    categorization_df[f'only_mes_relative_winners_avg_cost_{category}'] = categorization_df[f'only_mes_relative_winners_cost_pct_{category}'] / categorization_df[f'only_mes_relative_winners_count_pct_{category}']
    categorization_df[f'diff_go_mo_relative_winners_avg_cost_{category}'] = categorization_df[f'only_greedy_relative_winners_avg_cost_{category}'] - categorization_df[f'only_mes_relative_winners_avg_cost_{category}'] 

## Metrics for Cost and Project Representation

In [ ]:
# New metrics required relative proposals in each category
print("Before adding new metrics, shape was: ", categorization_df.shape)
for category in categories_set:
    categorization_df[f'greedy_relative_proposals_cost_pct_{category}'] = 1 * categorization_df[f'greedy_winners_{category}_total_cost'] / categorization_df[f'{category}_total_cost']
    categorization_df[f'mes_relative_proposals_cost_pct_{category}'] = 1 * categorization_df[f'mes_winners_{category}_total_cost'] / categorization_df[f'{category}_total_cost']
    categorization_df[f'diff_greedy_mes_relative_proposals_cost_pct_{category}'] = categorization_df[f'greedy_relative_proposals_cost_pct_{category}'] - categorization_df[f'mes_relative_proposals_cost_pct_{category}']
    
    categorization_df[f'greedy_relative_proposals_count_pct_{category}'] = 1 * categorization_df[f'greedy_winners_{category}_projects_count'] / categorization_df[f'{category}_projects_count']
    categorization_df[f'mes_relative_proposals_count_pct_{category}'] = 1 * categorization_df[f'mes_winners_{category}_projects_count'] / categorization_df[f'{category}_projects_count']
    categorization_df[f'diff_greedy_mes_relative_proposals_count_pct_{category}'] = categorization_df[f'greedy_relative_proposals_count_pct_{category}'] - categorization_df[f'mes_relative_proposals_count_pct_{category}']
    
    categorization_df[f'only_greedy_relative_proposals_cost_pct_{category}'] = 1 * categorization_df[f'only_greedy_winners_{category}_total_cost'] / categorization_df[f'{category}_total_cost']
    categorization_df[f'only_mes_relative_proposals_cost_pct_{category}'] = 1 * categorization_df[f'only_mes_winners_{category}_total_cost'] / categorization_df[f'{category}_total_cost']
    categorization_df[f'diff_go_mo_relative_proposals_cost_pct_{category}'] = categorization_df[f'only_greedy_relative_proposals_cost_pct_{category}'] - categorization_df[f'only_mes_relative_proposals_cost_pct_{category}']

    categorization_df[f'only_greedy_relative_proposals_count_pct_{category}'] = 1 * categorization_df[f'only_greedy_winners_{category}_projects_count'] / categorization_df[f'{category}_projects_count']
    categorization_df[f'only_mes_relative_proposals_count_pct_{category}'] = 1 * categorization_df[f'only_mes_winners_{category}_projects_count'] / categorization_df[f'{category}_projects_count']
    categorization_df[f'diff_go_mo_relative_proposals_count_pct_{category}'] = categorization_df[f'only_greedy_relative_proposals_count_pct_{category}'] - categorization_df[f'only_mes_relative_proposals_count_pct_{category}']

print("After adding new columns, shape should increase by 108 cols: ", categorization_df.shape)


In [ ]:
# additional metrics for average of relative proposals
for category in categories_set:
    categorization_df[f'greedy_relative_proposals_avg_cost_{category}'] = categorization_df[f'greedy_relative_proposals_cost_pct_{category}'] / categorization_df[f'greedy_relative_proposals_count_pct_{category}']
    categorization_df[f'mes_relative_proposals_avg_cost_{category}'] = categorization_df[f'mes_relative_proposals_cost_pct_{category}'] / categorization_df[f'mes_relative_proposals_count_pct_{category}']
    categorization_df[f'diff_ug_mes_relative_proposals_avg_cost_{category}'] = categorization_df[f'greedy_relative_proposals_avg_cost_{category}'] - categorization_df[f'mes_relative_proposals_avg_cost_{category}'] 

    categorization_df[f'only_greedy_relative_proposals_avg_cost_{category}'] = categorization_df[f'only_greedy_relative_proposals_cost_pct_{category}'] / categorization_df[f'only_greedy_relative_proposals_count_pct_{category}']
    categorization_df[f'only_mes_relative_proposals_avg_cost_{category}'] = categorization_df[f'only_mes_relative_proposals_cost_pct_{category}'] / categorization_df[f'only_mes_relative_proposals_count_pct_{category}']
    categorization_df[f'diff_go_mo_relative_proposals_avg_cost_{category}'] = categorization_df[f'only_greedy_relative_proposals_avg_cost_{category}'] - categorization_df[f'only_mes_relative_proposals_avg_cost_{category}'] 

## Metrics for Between and Within Cost and Count Novelty

In [ ]:
# initialize empty column values for novelty metrics
for category in categories_set:
    categorization_df[f'{category}_within_novelty_cost'] = 0
    categorization_df[f'{category}_within_novelty_count'] = 0
    categorization_df[f'{category}_between_novelty_cost'] = 0
    categorization_df[f'{category}_between_novelty_count'] = 0

# define functions that update those values
## within group novelty cost
def calculateWithinNoveltyCost(row):
    for category in categories_set:
        if (row[f'greedy_winners_{category}_total_cost'] == 0) and (row[f'mes_winners_{category}_total_cost'] != 0):
            row[f'{category}_within_novelty_cost'] = (0 - (1 * row[f'only_mes_winners_{category}_total_cost'] / row[f'mes_winners_{category}_total_cost']))
        elif (row[f'mes_winners_{category}_total_cost'] == 0) and (row[f'greedy_winners_{category}_total_cost'] != 0):
            row[f'{category}_within_novelty_cost'] = (1 * row[f'only_greedy_winners_{category}_total_cost'] / row[f'greedy_winners_{category}_total_cost'])
        elif (row[f'mes_winners_{category}_total_cost'] == 0) and (row[f'greedy_winners_{category}_total_cost'] == 0):
            row[f'{category}_within_novelty_cost'] = 0
        else:
            row[f'{category}_within_novelty_cost'] = ((1 * row[f'only_greedy_winners_{category}_total_cost'] / row[f'greedy_winners_{category}_total_cost']) - (1 * row[f'only_mes_winners_{category}_total_cost'] / row[f'mes_winners_{category}_total_cost']))
    return row


# within group novelty count
def calculateWithinNoveltyCount(row):
    for category in categories_set:
        if (row[f'greedy_winners_{category}_projects_count'] == 0) and (row[f'mes_winners_{category}_projects_count'] != 0):
            row[f'{category}_within_novelty_count'] = (0 - (1 * row[f'only_mes_winners_{category}_projects_count'] / row[f'mes_winners_{category}_projects_count']))
        elif (row[f'mes_winners_{category}_projects_count'] == 0) and (row[f'greedy_winners_{category}_projects_count'] != 0):
            row[f'{category}_within_novelty_count'] = (1 * row[f'only_greedy_winners_{category}_projects_count'] / row[f'greedy_winners_{category}_projects_count'])
        elif (row[f'mes_winners_{category}_projects_count'] == 0) and (row[f'greedy_winners_{category}_projects_count'] == 0):
            row[f'{category}_within_novelty_count'] = 0
        else:
            row[f'{category}_within_novelty_count'] = ((1 * row[f'only_greedy_winners_{category}_projects_count'] / row[f'greedy_winners_{category}_projects_count']) - (1 * row[f'only_mes_winners_{category}_projects_count'] / row[f'mes_winners_{category}_projects_count']))
    return row

# between novelty cost
def calculateBetweenNoveltyCost(row):
    for category in categories_set:
        if (row['only_greedy_winners_total_cost'] == 0) and (row['only_mes_winners_total_cost'] != 0):
            row[f'{category}_between_novelty_cost'] = (0 - (1 * row[f'only_mes_winners_{category}_total_cost'] / row['only_mes_winners_total_cost']))
        elif (row['only_mes_winners_total_cost'] == 0) and (row['only_greedy_winners_total_cost'] != 0):
            row[f'{category}_between_novelty_cost'] = (1 * row[f'only_greedy_winners_{category}_total_cost'] / row['only_greedy_winners_total_cost'])
        elif (row['only_mes_winners_total_cost'] == 0) and (row['only_greedy_winners_total_cost'] == 0):
            row[f'{category}_between_novelty_cost'] = 0
        else:
            row[f'{category}_between_novelty_cost'] = ((1 * row[f'only_greedy_winners_{category}_total_cost'] / row['only_greedy_winners_total_cost']) - (1 * row[f'only_mes_winners_{category}_total_cost'] / row['only_mes_winners_total_cost']))
    return row

# between groups novelty count
def calculateBetweenNoveltyCount(row):
    for category in categories_set:
        if (row['only_greedy_winners_projects_count'] == 0) and (row['only_mes_winners_projects_count'] != 0):
            row[f'{category}_between_novelty_count'] = (0 - (1 * row[f'only_mes_winners_{category}_projects_count'] / row['only_mes_winners_projects_count']))
        elif (row['only_greedy_winners_projects_count'] != 0) and (row['only_mes_winners_projects_count'] == 0):
            row[f'{category}_between_novelty_count'] = (1 * row[f'only_greedy_winners_{category}_projects_count'] / row['only_greedy_winners_projects_count'])
        elif (row['only_greedy_winners_projects_count'] == 0) and (row['only_mes_winners_projects_count'] == 0):
            row[f'{category}_between_novelty_count'] = 0
        else:
            row[f'{category}_between_novelty_count'] = ((1 * row[f'only_greedy_winners_{category}_projects_count'] / row['only_greedy_winners_projects_count']) - (1 * row[f'only_mes_winners_{category}_projects_count'] / row['only_mes_winners_projects_count']))
    return row

# iterate across all rows of categorization_df and update the values
categorization_df = categorization_df.apply(lambda row: calculateWithinNoveltyCost(row), axis=1)
categorization_df = categorization_df.apply(lambda row: calculateWithinNoveltyCount(row), axis=1)
categorization_df = categorization_df.apply(lambda row: calculateBetweenNoveltyCost(row), axis=1)
categorization_df = categorization_df.apply(lambda row: calculateBetweenNoveltyCount(row), axis=1)

## Hacking the dataset to select only instances for which there are exclusive winning outcomes by either MES or UG at each category level

In [ ]:
for category in categories_set:
    cols_to_select = ['election_id', 
                            f'greedy_relative_winners_cost_pct_{category}', 
                            f'mes_relative_winners_cost_pct_{category}', 
                            f'diff_greedy_mes_relative_winners_cost_pct_{category}', 
                            f'greedy_relative_winners_count_pct_{category}', 
                            f'mes_relative_winners_count_pct_{category}', 
                            f'diff_greedy_mes_relative_winners_count_pct_{category}', 
                            f'only_greedy_relative_winners_cost_pct_{category}', 
                            f'only_mes_relative_winners_cost_pct_{category}',
                            f'diff_go_mo_relative_winners_cost_pct_{category}',
                            f'only_greedy_relative_winners_count_pct_{category}',
                            f'only_mes_relative_winners_count_pct_{category}',
                            f'diff_go_mo_relative_winners_count_pct_{category}',

                            f'greedy_relative_proposals_cost_pct_{category}',
                            f'mes_relative_proposals_cost_pct_{category}',
                            f'diff_greedy_mes_relative_proposals_cost_pct_{category}',
                            f'greedy_relative_proposals_count_pct_{category}',
                            f'mes_relative_proposals_count_pct_{category}',
                            f'diff_greedy_mes_relative_proposals_count_pct_{category}',
                            f'only_greedy_relative_proposals_cost_pct_{category}',
                            f'only_mes_relative_proposals_cost_pct_{category}',
                            f'diff_go_mo_relative_proposals_cost_pct_{category}',
                            f'only_greedy_relative_proposals_count_pct_{category}',
                            f'only_mes_relative_proposals_count_pct_{category}',
                            f'diff_go_mo_relative_proposals_count_pct_{category}', 

                            # additional columns for novelty measure
                            f'{category}_within_novelty_cost',
                            f'{category}_within_novelty_count',
                            f'{category}_between_novelty_cost',
                            f'{category}_between_novelty_count'
                        ]
    if category == 'education':
        education_df = categorization_df[cols_to_select]
        education_df.drop_duplicates(inplace=True)
        print(education_df.shape)
    if category == 'public_transit_and_roads':
        public_transit_and_roads_df = categorization_df[cols_to_select]
        public_transit_and_roads_df.drop_duplicates(inplace=True)
        print(public_transit_and_roads_df.shape)
    if category == 'health':
        health_df = categorization_df[cols_to_select]
        health_df.drop_duplicates(inplace=True)
        print(health_df.shape)
    if category == 'welfare':
        welfare_df = categorization_df[cols_to_select]
        welfare_df.drop_duplicates(inplace=True)
        print(welfare_df.shape)
    if category == 'public_space':
        public_space_df = categorization_df[cols_to_select]
        public_space_df.drop_duplicates(inplace=True)
        print(public_space_df.shape)
    if category == 'urban_greenery':
        urban_greenery_df = categorization_df[cols_to_select]
        urban_greenery_df.drop_duplicates(inplace=True)
        print(urban_greenery_df.shape)
    if category == 'culture':
        culture_df = categorization_df[cols_to_select]
        culture_df.drop_duplicates(inplace=True)
        print(culture_df.shape)
    if category == 'sport':
        sport_df = categorization_df[cols_to_select]
        sport_df.drop_duplicates(inplace=True)
        print(sport_df.shape)
    if category == 'env_protection':
        env_protection = categorization_df[cols_to_select]
        env_protection.drop_duplicates(inplace=True)
        print(env_protection.shape)

In [ ]:
for category in categories_set:
    bs_cols_for_plot = [
                        'election_id', 
                        f'greedy_relative_winners_cost_pct_{category}', 
                        f'mes_relative_winners_cost_pct_{category}', 
                        f'diff_greedy_mes_relative_winners_cost_pct_{category}', 
                        f'only_greedy_relative_winners_cost_pct_{category}', 
                        f'only_mes_relative_winners_cost_pct_{category}',
                        f'diff_go_mo_relative_winners_cost_pct_{category}',

                        f'{category}_within_novelty_cost',
                        f'{category}_between_novelty_cost'
                    ]

    wr_cols_for_plot = [
                        'election_id', 
                        f'greedy_relative_winners_count_pct_{category}', 
                        f'mes_relative_winners_count_pct_{category}', 
                        f'diff_greedy_mes_relative_winners_count_pct_{category}', 
                        f'only_greedy_relative_winners_count_pct_{category}',
                        f'only_mes_relative_winners_count_pct_{category}',
                        f'diff_go_mo_relative_winners_count_pct_{category}',

                        f'{category}_within_novelty_count',
                        f'{category}_between_novelty_count'
                    ]

    cr_cols_for_plot = [
                        'election_id', 
                        f'greedy_relative_proposals_cost_pct_{category}',
                        f'mes_relative_proposals_cost_pct_{category}',
                        f'diff_greedy_mes_relative_proposals_cost_pct_{category}',
                        f'only_greedy_relative_proposals_cost_pct_{category}',
                        f'only_mes_relative_proposals_cost_pct_{category}',
                        f'diff_go_mo_relative_proposals_cost_pct_{category}',

                        f'{category}_within_novelty_cost',
                        f'{category}_between_novelty_cost'
                    ]

    pr_cols_for_plot = [
                        'election_id', 
                        f'greedy_relative_proposals_count_pct_{category}',
                        f'mes_relative_proposals_count_pct_{category}',
                        f'diff_greedy_mes_relative_proposals_count_pct_{category}',
                        f'only_greedy_relative_proposals_count_pct_{category}',
                        f'only_mes_relative_proposals_count_pct_{category}',
                        f'diff_go_mo_relative_proposals_count_pct_{category}',

                        f'{category}_within_novelty_count',
                        f'{category}_between_novelty_count'
                    ]
    
    if category == 'education':
        education_bs_df = education_df[bs_cols_for_plot]
        education_bs_df = education_bs_df[(education_bs_df[f'only_greedy_relative_winners_cost_pct_{category}'] > 0) | (education_bs_df[f'only_mes_relative_winners_cost_pct_{category}'] > 0)]
        education_bs_df.rename(columns={
            f'greedy_relative_winners_cost_pct_{category}': 'ug_bs',
            f'mes_relative_winners_cost_pct_{category}': 'mes_bs',
            f'only_greedy_relative_winners_cost_pct_{category}': 'excl_ug_bs',
            f'only_mes_relative_winners_cost_pct_{category}': 'excl_mes_bs',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        education_bs_df['category'] = 'Education'
        
        education_wr_df = education_df[wr_cols_for_plot]
        education_wr_df = education_wr_df[(education_wr_df[f'only_greedy_relative_winners_count_pct_{category}'] > 0) | (education_wr_df[f'only_mes_relative_winners_count_pct_{category}'] > 0)]
        education_wr_df.rename(columns={
            f'greedy_relative_winners_count_pct_{category}': 'ug_wr',
            f'mes_relative_winners_count_pct_{category}': 'mes_wr',
            f'only_greedy_relative_winners_count_pct_{category}': 'excl_ug_wr',
            f'only_mes_relative_winners_count_pct_{category}': 'excl_mes_wr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        education_wr_df['category'] = 'Education'

        education_cr_df = education_df[cr_cols_for_plot]
        education_cr_df = education_cr_df[(education_cr_df[f'only_greedy_relative_proposals_cost_pct_{category}'] > 0) | (education_cr_df[f'only_mes_relative_proposals_cost_pct_{category}'] > 0)]
        education_cr_df.rename(columns={
            f'greedy_relative_proposals_cost_pct_{category}': 'ug_cr',
            f'mes_relative_proposals_cost_pct_{category}': 'mes_cr',
            f'only_greedy_relative_proposals_cost_pct_{category}': 'excl_ug_cr',
            f'only_mes_relative_proposals_cost_pct_{category}': 'excl_mes_cr',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        education_cr_df['category'] = 'Education'

        education_pr_df = education_df[pr_cols_for_plot]
        education_pr_df = education_pr_df[(education_pr_df[f'only_greedy_relative_proposals_count_pct_{category}'] > 0) | (education_pr_df[f'only_mes_relative_proposals_count_pct_{category}'] > 0)]
        education_pr_df.rename(columns={
            f'greedy_relative_proposals_count_pct_{category}': 'ug_pr',
            f'mes_relative_proposals_count_pct_{category}': 'mes_pr',
            f'only_greedy_relative_proposals_count_pct_{category}': 'excl_ug_pr',
            f'only_mes_relative_proposals_count_pct_{category}': 'excl_mes_pr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        education_pr_df['category'] = 'Education'

    if category == 'public_transit_and_roads':
        public_transit_and_roads_bs_df = public_transit_and_roads_df[bs_cols_for_plot]
        public_transit_and_roads_bs_df = public_transit_and_roads_bs_df[(public_transit_and_roads_bs_df[f'only_greedy_relative_winners_cost_pct_{category}'] > 0) | (public_transit_and_roads_bs_df[f'only_mes_relative_winners_cost_pct_{category}'] > 0)]
        public_transit_and_roads_bs_df.rename(columns={
            f'greedy_relative_winners_cost_pct_{category}': 'ug_bs',
            f'mes_relative_winners_cost_pct_{category}': 'mes_bs',
            f'only_greedy_relative_winners_cost_pct_{category}': 'excl_ug_bs',
            f'only_mes_relative_winners_cost_pct_{category}': 'excl_mes_bs',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        public_transit_and_roads_bs_df['category'] = 'Public\n Transit'
        
        public_transit_and_roads_wr_df = public_transit_and_roads_df[wr_cols_for_plot]
        public_transit_and_roads_wr_df = public_transit_and_roads_wr_df[(public_transit_and_roads_wr_df[f'only_greedy_relative_winners_count_pct_{category}'] > 0) | (public_transit_and_roads_wr_df[f'only_mes_relative_winners_count_pct_{category}'] > 0)]
        public_transit_and_roads_wr_df.rename(columns={
            f'greedy_relative_winners_count_pct_{category}': 'ug_wr',
            f'mes_relative_winners_count_pct_{category}': 'mes_wr',
            f'only_greedy_relative_winners_count_pct_{category}': 'excl_ug_wr',
            f'only_mes_relative_winners_count_pct_{category}': 'excl_mes_wr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        public_transit_and_roads_wr_df['category'] = 'Public\n Transit'
        
        public_transit_and_roads_cr_df = public_transit_and_roads_df[cr_cols_for_plot]
        public_transit_and_roads_cr_df = public_transit_and_roads_cr_df[(public_transit_and_roads_cr_df[f'only_greedy_relative_proposals_cost_pct_{category}'] > 0) | (public_transit_and_roads_cr_df[f'only_mes_relative_proposals_cost_pct_{category}'] > 0)]
        public_transit_and_roads_cr_df.rename(columns={
            f'greedy_relative_proposals_cost_pct_{category}': 'ug_cr',
            f'mes_relative_proposals_cost_pct_{category}': 'mes_cr',
            f'only_greedy_relative_proposals_cost_pct_{category}': 'excl_ug_cr',
            f'only_mes_relative_proposals_cost_pct_{category}': 'excl_mes_cr',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        public_transit_and_roads_cr_df['category'] = 'Public\n Transit'

        public_transit_and_roads_pr_df = public_transit_and_roads_df[pr_cols_for_plot]
        public_transit_and_roads_pr_df = public_transit_and_roads_pr_df[(public_transit_and_roads_pr_df[f'only_greedy_relative_proposals_count_pct_{category}'] > 0) | (public_transit_and_roads_pr_df[f'only_mes_relative_proposals_count_pct_{category}'] > 0)]
        public_transit_and_roads_pr_df.rename(columns={
            f'greedy_relative_proposals_count_pct_{category}': 'ug_pr',
            f'mes_relative_proposals_count_pct_{category}': 'mes_pr',
            f'only_greedy_relative_proposals_count_pct_{category}': 'excl_ug_pr',
            f'only_mes_relative_proposals_count_pct_{category}': 'excl_mes_pr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        public_transit_and_roads_pr_df['category'] = 'Public\n Transit'
    
    if category == 'welfare':
        welfare_bs_df = welfare_df[bs_cols_for_plot]
        welfare_bs_df = welfare_bs_df[(welfare_bs_df[f'only_greedy_relative_winners_cost_pct_{category}'] > 0) | (welfare_bs_df[f'only_mes_relative_winners_cost_pct_{category}'] > 0)]
        welfare_bs_df.rename(columns={
            f'greedy_relative_winners_cost_pct_{category}': 'ug_bs',
            f'mes_relative_winners_cost_pct_{category}': 'mes_bs',
            f'only_greedy_relative_winners_cost_pct_{category}': 'excl_ug_bs',
            f'only_mes_relative_winners_cost_pct_{category}': 'excl_mes_bs',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        welfare_bs_df['category'] = 'Welfare'
        
        welfare_wr_df = welfare_df[wr_cols_for_plot]
        welfare_wr_df = welfare_wr_df[(welfare_wr_df[f'only_greedy_relative_winners_count_pct_{category}'] > 0) | (welfare_wr_df[f'only_mes_relative_winners_count_pct_{category}'] > 0)]
        welfare_wr_df.rename(columns={
            f'greedy_relative_winners_count_pct_{category}': 'ug_wr',
            f'mes_relative_winners_count_pct_{category}': 'mes_wr',
            f'only_greedy_relative_winners_count_pct_{category}': 'excl_ug_wr',
            f'only_mes_relative_winners_count_pct_{category}': 'excl_mes_wr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        welfare_wr_df['category'] = 'Welfare'

        welfare_cr_df = welfare_df[cr_cols_for_plot]
        welfare_cr_df = welfare_cr_df[(welfare_cr_df[f'only_greedy_relative_proposals_cost_pct_{category}'] > 0) | (welfare_cr_df[f'only_mes_relative_proposals_cost_pct_{category}'] > 0)]
        welfare_cr_df.rename(columns={
            f'greedy_relative_proposals_cost_pct_{category}': 'ug_cr',
            f'mes_relative_proposals_cost_pct_{category}': 'mes_cr',
            f'only_greedy_relative_proposals_cost_pct_{category}': 'excl_ug_cr',
            f'only_mes_relative_proposals_cost_pct_{category}': 'excl_mes_cr',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        welfare_cr_df['category'] = 'Welfare'

        welfare_pr_df = welfare_df[pr_cols_for_plot]
        welfare_pr_df = welfare_pr_df[(welfare_pr_df[f'only_greedy_relative_proposals_count_pct_{category}'] > 0) | (welfare_pr_df[f'only_mes_relative_proposals_count_pct_{category}'] > 0)]
        welfare_pr_df.rename(columns={
            f'greedy_relative_proposals_count_pct_{category}': 'ug_pr',
            f'mes_relative_proposals_count_pct_{category}': 'mes_pr',
            f'only_greedy_relative_proposals_count_pct_{category}': 'excl_ug_pr',
            f'only_mes_relative_proposals_count_pct_{category}': 'excl_mes_pr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        welfare_pr_df['category'] = 'Welfare'

    if category == 'health':
        health_bs_df = health_df[bs_cols_for_plot]
        health_bs_df = health_bs_df[(health_bs_df[f'only_greedy_relative_winners_cost_pct_{category}'] > 0) | (health_bs_df[f'only_mes_relative_winners_cost_pct_{category}'] > 0)]
        health_bs_df.rename(columns={
            f'greedy_relative_winners_cost_pct_{category}': 'ug_bs',
            f'mes_relative_winners_cost_pct_{category}': 'mes_bs',
            f'only_greedy_relative_winners_cost_pct_{category}': 'excl_ug_bs',
            f'only_mes_relative_winners_cost_pct_{category}': 'excl_mes_bs',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        health_bs_df['category'] = 'Health'
        
        health_wr_df = health_df[wr_cols_for_plot]
        health_wr_df = health_wr_df[(health_wr_df[f'only_greedy_relative_winners_count_pct_{category}'] > 0) | (health_wr_df[f'only_mes_relative_winners_count_pct_{category}'] > 0)]
        health_wr_df.rename(columns={
            f'greedy_relative_winners_count_pct_{category}': 'ug_wr',
            f'mes_relative_winners_count_pct_{category}': 'mes_wr',
            f'only_greedy_relative_winners_count_pct_{category}': 'excl_ug_wr',
            f'only_mes_relative_winners_count_pct_{category}': 'excl_mes_wr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        health_wr_df['category'] = 'Health'

        health_cr_df = health_df[cr_cols_for_plot]
        health_cr_df = health_cr_df[(health_cr_df[f'only_greedy_relative_proposals_cost_pct_{category}'] > 0) | (health_cr_df[f'only_mes_relative_proposals_cost_pct_{category}'] > 0)]
        health_cr_df.rename(columns={
            f'greedy_relative_proposals_cost_pct_{category}': 'ug_cr',
            f'mes_relative_proposals_cost_pct_{category}': 'mes_cr',
            f'only_greedy_relative_proposals_cost_pct_{category}': 'excl_ug_cr',
            f'only_mes_relative_proposals_cost_pct_{category}': 'excl_mes_cr',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        health_cr_df['category'] = 'Health'

        health_pr_df = health_df[pr_cols_for_plot]
        health_pr_df = health_pr_df[(health_pr_df[f'only_greedy_relative_proposals_count_pct_{category}'] > 0) | (health_pr_df[f'only_mes_relative_proposals_count_pct_{category}'] > 0)]
        health_pr_df.rename(columns={
            f'greedy_relative_proposals_count_pct_{category}': 'ug_pr',
            f'mes_relative_proposals_count_pct_{category}': 'mes_pr',
            f'only_greedy_relative_proposals_count_pct_{category}': 'excl_ug_pr',
            f'only_mes_relative_proposals_count_pct_{category}': 'excl_mes_pr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        health_pr_df['category'] = 'Health'
    
    if category == 'public_space':
        public_space_bs_df = public_space_df[bs_cols_for_plot]
        public_space_bs_df = public_space_bs_df[(public_space_bs_df[f'only_greedy_relative_winners_cost_pct_{category}'] > 0) | (public_space_bs_df[f'only_mes_relative_winners_cost_pct_{category}'] > 0)]
        public_space_bs_df.rename(columns={
            f'greedy_relative_winners_cost_pct_{category}': 'ug_bs',
            f'mes_relative_winners_cost_pct_{category}': 'mes_bs',
            f'only_greedy_relative_winners_cost_pct_{category}': 'excl_ug_bs',
            f'only_mes_relative_winners_cost_pct_{category}': 'excl_mes_bs',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        public_space_bs_df['category'] = 'Public\n Space'
        
        public_space_wr_df = public_space_df[wr_cols_for_plot]
        public_space_wr_df = public_space_wr_df[(public_space_wr_df[f'only_greedy_relative_winners_count_pct_{category}'] > 0) | (public_space_wr_df[f'only_mes_relative_winners_count_pct_{category}'] > 0)]
        public_space_wr_df.rename(columns={
            f'greedy_relative_winners_count_pct_{category}': 'ug_wr',
            f'mes_relative_winners_count_pct_{category}': 'mes_wr',
            f'only_greedy_relative_winners_count_pct_{category}': 'excl_ug_wr',
            f'only_mes_relative_winners_count_pct_{category}': 'excl_mes_wr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        public_space_wr_df['category'] = 'Public\n Space'
        
        public_space_cr_df = public_space_df[cr_cols_for_plot]
        public_space_cr_df = public_space_cr_df[(public_space_cr_df[f'only_greedy_relative_proposals_cost_pct_{category}'] > 0) | (public_space_cr_df[f'only_mes_relative_proposals_cost_pct_{category}'] > 0)]
        public_space_cr_df.rename(columns={
            f'greedy_relative_proposals_cost_pct_{category}': 'ug_cr',
            f'mes_relative_proposals_cost_pct_{category}': 'mes_cr',
            f'only_greedy_relative_proposals_cost_pct_{category}': 'excl_ug_cr',
            f'only_mes_relative_proposals_cost_pct_{category}': 'excl_mes_cr',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        public_space_cr_df['category'] = 'Public\n Space'

        public_space_pr_df = public_space_df[pr_cols_for_plot]
        public_space_pr_df = public_space_pr_df[(public_space_pr_df[f'only_greedy_relative_proposals_count_pct_{category}'] > 0) | (public_space_pr_df[f'only_mes_relative_proposals_count_pct_{category}'] > 0)]
        public_space_pr_df.rename(columns={
            f'greedy_relative_proposals_count_pct_{category}': 'ug_pr',
            f'mes_relative_proposals_count_pct_{category}': 'mes_pr',
            f'only_greedy_relative_proposals_count_pct_{category}': 'excl_ug_pr',
            f'only_mes_relative_proposals_count_pct_{category}': 'excl_mes_pr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        public_space_pr_df['category'] = 'Public\n Space'

    if category == 'urban_greenery':
        urban_greenery_bs_df = urban_greenery_df[bs_cols_for_plot]
        urban_greenery_bs_df = urban_greenery_bs_df[(urban_greenery_bs_df[f'only_greedy_relative_winners_cost_pct_{category}'] > 0) | (urban_greenery_bs_df[f'only_mes_relative_winners_cost_pct_{category}'] > 0)]
        urban_greenery_bs_df.rename(columns={
            f'greedy_relative_winners_cost_pct_{category}': 'ug_bs',
            f'mes_relative_winners_cost_pct_{category}': 'mes_bs',
            f'only_greedy_relative_winners_cost_pct_{category}': 'excl_ug_bs',
            f'only_mes_relative_winners_cost_pct_{category}': 'excl_mes_bs',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        urban_greenery_bs_df['category'] = 'Urban\n Greenery'
        
        urban_greenery_wr_df = urban_greenery_df[wr_cols_for_plot]
        urban_greenery_wr_df = urban_greenery_wr_df[(urban_greenery_wr_df[f'only_greedy_relative_winners_count_pct_{category}'] > 0) | (urban_greenery_wr_df[f'only_mes_relative_winners_count_pct_{category}'] > 0)]
        urban_greenery_wr_df.rename(columns={
            f'greedy_relative_winners_count_pct_{category}': 'ug_wr',
            f'mes_relative_winners_count_pct_{category}': 'mes_wr',
            f'only_greedy_relative_winners_count_pct_{category}': 'excl_ug_wr',
            f'only_mes_relative_winners_count_pct_{category}': 'excl_mes_wr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        urban_greenery_wr_df['category'] = 'Urban\n Greenery'

        urban_greenery_cr_df = urban_greenery_df[cr_cols_for_plot]
        urban_greenery_cr_df = urban_greenery_cr_df[(urban_greenery_cr_df[f'only_greedy_relative_proposals_cost_pct_{category}'] > 0) | (urban_greenery_cr_df[f'only_mes_relative_proposals_cost_pct_{category}'] > 0)]
        urban_greenery_cr_df.rename(columns={
            f'greedy_relative_proposals_cost_pct_{category}': 'ug_cr',
            f'mes_relative_proposals_cost_pct_{category}': 'mes_cr',
            f'only_greedy_relative_proposals_cost_pct_{category}': 'excl_ug_cr',
            f'only_mes_relative_proposals_cost_pct_{category}': 'excl_mes_cr',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        urban_greenery_cr_df['category'] = 'Urban\n Greenery'

        urban_greenery_pr_df = urban_greenery_df[pr_cols_for_plot]
        urban_greenery_pr_df = urban_greenery_pr_df[(urban_greenery_pr_df[f'only_greedy_relative_proposals_count_pct_{category}'] > 0) | (urban_greenery_pr_df[f'only_mes_relative_proposals_count_pct_{category}'] > 0)]
        urban_greenery_pr_df.rename(columns={
            f'greedy_relative_proposals_count_pct_{category}': 'ug_pr',
            f'mes_relative_proposals_count_pct_{category}': 'mes_pr',
            f'only_greedy_relative_proposals_count_pct_{category}': 'excl_ug_pr',
            f'only_mes_relative_proposals_count_pct_{category}': 'excl_mes_pr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        urban_greenery_pr_df['category'] = 'Urban\n Greenery'

    if category == 'culture':
        culture_bs_df = culture_df[bs_cols_for_plot]
        culture_bs_df = culture_bs_df[(culture_bs_df[f'only_greedy_relative_winners_cost_pct_{category}'] > 0) | (culture_bs_df[f'only_mes_relative_winners_cost_pct_{category}'] > 0)]
        culture_bs_df.rename(columns={
            f'greedy_relative_winners_cost_pct_{category}': 'ug_bs',
            f'mes_relative_winners_cost_pct_{category}': 'mes_bs',
            f'only_greedy_relative_winners_cost_pct_{category}': 'excl_ug_bs',
            f'only_mes_relative_winners_cost_pct_{category}': 'excl_mes_bs',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        culture_bs_df['category'] = 'Culture'
        
        culture_wr_df = culture_df[wr_cols_for_plot]
        culture_wr_df = culture_wr_df[(culture_wr_df[f'only_greedy_relative_winners_count_pct_{category}'] > 0) | (culture_wr_df[f'only_mes_relative_winners_count_pct_{category}'] > 0)]
        culture_wr_df.rename(columns={
            f'greedy_relative_winners_count_pct_{category}': 'ug_wr',
            f'mes_relative_winners_count_pct_{category}': 'mes_wr',
            f'only_greedy_relative_winners_count_pct_{category}': 'excl_ug_wr',
            f'only_mes_relative_winners_count_pct_{category}': 'excl_mes_wr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        culture_wr_df['category'] = 'Culture'

        culture_cr_df = culture_df[cr_cols_for_plot]
        culture_cr_df = culture_cr_df[(culture_cr_df[f'only_greedy_relative_proposals_cost_pct_{category}'] > 0) | (culture_cr_df[f'only_mes_relative_proposals_cost_pct_{category}'] > 0)]
        culture_cr_df.rename(columns={
            f'greedy_relative_proposals_cost_pct_{category}': 'ug_cr',
            f'mes_relative_proposals_cost_pct_{category}': 'mes_cr',
            f'only_greedy_relative_proposals_cost_pct_{category}': 'excl_ug_cr',
            f'only_mes_relative_proposals_cost_pct_{category}': 'excl_mes_cr',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        culture_cr_df['category'] = 'Culture'

        culture_pr_df = culture_df[pr_cols_for_plot]
        culture_pr_df = culture_pr_df[(culture_pr_df[f'only_greedy_relative_proposals_count_pct_{category}'] > 0) | (culture_pr_df[f'only_mes_relative_proposals_count_pct_{category}'] > 0)]
        culture_pr_df.rename(columns={
            f'greedy_relative_proposals_count_pct_{category}': 'ug_pr',
            f'mes_relative_proposals_count_pct_{category}': 'mes_pr',
            f'only_greedy_relative_proposals_count_pct_{category}': 'excl_ug_pr',
            f'only_mes_relative_proposals_count_pct_{category}': 'excl_mes_pr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        culture_pr_df['category'] = 'Culture'

    if category == 'sport':
        sport_bs_df = sport_df[bs_cols_for_plot]
        sport_bs_df = sport_bs_df[(sport_bs_df[f'only_greedy_relative_winners_cost_pct_{category}'] > 0) | (sport_bs_df[f'only_mes_relative_winners_cost_pct_{category}'] > 0)]
        sport_bs_df.rename(columns={
            f'greedy_relative_winners_cost_pct_{category}': 'ug_bs',
            f'mes_relative_winners_cost_pct_{category}': 'mes_bs',
            f'only_greedy_relative_winners_cost_pct_{category}': 'excl_ug_bs',
            f'only_mes_relative_winners_cost_pct_{category}': 'excl_mes_bs',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        sport_bs_df['category'] = 'Sport'
        
        sport_wr_df = sport_df[wr_cols_for_plot]
        sport_wr_df = sport_wr_df[(sport_wr_df[f'only_greedy_relative_winners_count_pct_{category}'] > 0) | (sport_wr_df[f'only_mes_relative_winners_count_pct_{category}'] > 0)]
        sport_wr_df.rename(columns={
            f'greedy_relative_winners_count_pct_{category}': 'ug_wr',
            f'mes_relative_winners_count_pct_{category}': 'mes_wr',
            f'only_greedy_relative_winners_count_pct_{category}': 'excl_ug_wr',
            f'only_mes_relative_winners_count_pct_{category}': 'excl_mes_wr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        sport_wr_df['category'] = 'Sport'

        sport_cr_df = sport_df[cr_cols_for_plot]
        sport_cr_df = sport_cr_df[(sport_cr_df[f'only_greedy_relative_proposals_cost_pct_{category}'] > 0) | (sport_cr_df[f'only_mes_relative_proposals_cost_pct_{category}'] > 0)]
        sport_cr_df.rename(columns={
            f'greedy_relative_proposals_cost_pct_{category}': 'ug_cr',
            f'mes_relative_proposals_cost_pct_{category}': 'mes_cr',
            f'only_greedy_relative_proposals_cost_pct_{category}': 'excl_ug_cr',
            f'only_mes_relative_proposals_cost_pct_{category}': 'excl_mes_cr',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        sport_cr_df['category'] = 'Sport'

        sport_pr_df = sport_df[pr_cols_for_plot]
        sport_pr_df = sport_pr_df[(sport_pr_df[f'only_greedy_relative_proposals_count_pct_{category}'] > 0) | (sport_pr_df[f'only_mes_relative_proposals_count_pct_{category}'] > 0)]
        sport_pr_df.rename(columns={
            f'greedy_relative_proposals_count_pct_{category}': 'ug_pr',
            f'mes_relative_proposals_count_pct_{category}': 'mes_pr',
            f'only_greedy_relative_proposals_count_pct_{category}': 'excl_ug_pr',
            f'only_mes_relative_proposals_count_pct_{category}': 'excl_mes_pr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        sport_pr_df['category'] = 'Sport'

    if category == 'env_protection':
        env_protection_bs_df = env_protection[bs_cols_for_plot]
        env_protection_bs_df = env_protection_bs_df[(env_protection_bs_df[f'only_greedy_relative_winners_cost_pct_{category}'] > 0) | (env_protection_bs_df[f'only_mes_relative_winners_cost_pct_{category}'] > 0)]
        env_protection_bs_df.rename(columns={
            f'greedy_relative_winners_cost_pct_{category}': 'ug_bs',
            f'mes_relative_winners_cost_pct_{category}': 'mes_bs',
            f'only_greedy_relative_winners_cost_pct_{category}': 'excl_ug_bs',
            f'only_mes_relative_winners_cost_pct_{category}': 'excl_mes_bs',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        env_protection_bs_df['category'] = 'Environmental\n Protection'
        
        env_protection_wr_df = env_protection[wr_cols_for_plot]
        env_protection_wr_df = env_protection_wr_df[(env_protection_wr_df[f'only_greedy_relative_winners_count_pct_{category}'] > 0) | (env_protection_wr_df[f'only_mes_relative_winners_count_pct_{category}'] > 0)]
        env_protection_wr_df.rename(columns={
            f'greedy_relative_winners_count_pct_{category}': 'ug_wr',
            f'mes_relative_winners_count_pct_{category}': 'mes_wr',
            f'only_greedy_relative_winners_count_pct_{category}': 'excl_ug_wr',
            f'only_mes_relative_winners_count_pct_{category}': 'excl_mes_wr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        env_protection_wr_df['category'] = 'Environmental\n Protection'

        env_protection_cr_df = env_protection[cr_cols_for_plot]
        env_protection_cr_df = env_protection_cr_df[(env_protection_cr_df[f'only_greedy_relative_proposals_cost_pct_{category}'] > 0) | (env_protection_cr_df[f'only_mes_relative_proposals_cost_pct_{category}'] > 0)]
        env_protection_cr_df.rename(columns={
            f'greedy_relative_proposals_cost_pct_{category}': 'ug_cr',
            f'mes_relative_proposals_cost_pct_{category}': 'mes_cr',
            f'only_greedy_relative_proposals_cost_pct_{category}': 'excl_ug_cr',
            f'only_mes_relative_proposals_cost_pct_{category}': 'excl_mes_cr',
            f'{category}_within_novelty_cost': 'within_novelty',
            f'{category}_between_novelty_cost': 'between_novelty'
        }, inplace=True)
        env_protection_cr_df['category'] = 'Environmental\n Protection'

        env_protection_pr_df = env_protection[pr_cols_for_plot]
        env_protection_pr_df = env_protection_pr_df[(env_protection_pr_df[f'only_greedy_relative_proposals_count_pct_{category}'] > 0) | (env_protection_pr_df[f'only_mes_relative_proposals_count_pct_{category}'] > 0)]
        env_protection_pr_df.rename(columns={
            f'greedy_relative_proposals_count_pct_{category}': 'ug_pr',
            f'mes_relative_proposals_count_pct_{category}': 'mes_pr',
            f'only_greedy_relative_proposals_count_pct_{category}': 'excl_ug_pr',
            f'only_mes_relative_proposals_count_pct_{category}': 'excl_mes_pr',
            f'{category}_within_novelty_count': 'within_novelty',
            f'{category}_between_novelty_count': 'between_novelty'
        }, inplace=True)
        env_protection_pr_df['category'] = 'Environmental\n Protection'

In [ ]:
## introducting impact metric value from novelty of exclusiveness for budget share
def calculateBSNoveltyImpact(row):
    if row['ug_bs'] == 0:
        return (0 - (1 * row['excl_mes_bs'] / row['mes_bs']))
    elif row['mes_bs'] == 0:
        return (1 * row['excl_ug_bs'] / row['ug_bs'])
    else:
        return ((1 * row['excl_ug_bs'] / row['ug_bs']) - (1 * row['excl_mes_bs'] / row['mes_bs']))
    
education_bs_df['novelty_impact_rate'] = education_bs_df.apply(calculateBSNoveltyImpact, axis=1)
health_bs_df['novelty_impact_rate'] = health_bs_df.apply(calculateBSNoveltyImpact, axis=1)
culture_bs_df['novelty_impact_rate'] = culture_bs_df.apply(calculateBSNoveltyImpact, axis=1)
public_space_bs_df['novelty_impact_rate'] = public_space_bs_df.apply(calculateBSNoveltyImpact, axis=1)
welfare_bs_df['novelty_impact_rate'] = welfare_bs_df.apply(calculateBSNoveltyImpact, axis=1)
sport_bs_df['novelty_impact_rate'] = sport_bs_df.apply(calculateBSNoveltyImpact, axis=1)
public_transit_and_roads_bs_df['novelty_impact_rate'] = public_transit_and_roads_bs_df.apply(calculateBSNoveltyImpact, axis=1)
urban_greenery_bs_df['novelty_impact_rate'] = urban_greenery_bs_df.apply(calculateBSNoveltyImpact, axis=1)
env_protection_bs_df['novelty_impact_rate'] = env_protection_bs_df.apply(calculateBSNoveltyImpact, axis=1)

In [ ]:
## introducting impact metric value from novelty of exclusiveness for winning rate
def calculateWRNoveltyImpact(row):
    if row['ug_wr'] == 0:
        return (0 - (1 * row['excl_mes_wr'] / row['mes_wr']))
    elif row['mes_wr'] == 0:
        return (1 * row['excl_ug_wr'] / row['ug_wr'])
    else:
        return ((1 * row['excl_ug_wr'] / row['ug_wr']) - (1 * row['excl_mes_wr'] / row['mes_wr']))
    
education_wr_df['novelty_impact_rate'] = education_wr_df.apply(calculateWRNoveltyImpact, axis=1)
health_wr_df['novelty_impact_rate'] = health_wr_df.apply(calculateWRNoveltyImpact, axis=1)
culture_wr_df['novelty_impact_rate'] = culture_wr_df.apply(calculateWRNoveltyImpact, axis=1)
public_space_wr_df['novelty_impact_rate'] = public_space_wr_df.apply(calculateWRNoveltyImpact, axis=1)
welfare_wr_df['novelty_impact_rate'] = welfare_wr_df.apply(calculateWRNoveltyImpact, axis=1)
sport_wr_df['novelty_impact_rate'] = sport_wr_df.apply(calculateWRNoveltyImpact, axis=1)
public_transit_and_roads_wr_df['novelty_impact_rate'] = public_transit_and_roads_wr_df.apply(calculateWRNoveltyImpact, axis=1)
urban_greenery_wr_df['novelty_impact_rate'] = urban_greenery_wr_df.apply(calculateWRNoveltyImpact, axis=1)
env_protection_wr_df['novelty_impact_rate'] = env_protection_wr_df.apply(calculateWRNoveltyImpact, axis=1)

In [ ]:
## introducing impact metric value from novelty of exclusiveness for cost representation
def calculateCRNoveltyImpact(row):
    if row['ug_cr'] == 0:
        return (0 - (1 * row['excl_mes_cr'] / row['mes_cr']))
    elif row['mes_cr'] == 0:
        return (1 * row['excl_ug_cr'] / row['ug_cr'])
    else:
        return ((1 * row['excl_ug_cr'] / row['ug_cr']) - (1 * row['excl_mes_cr'] / row['mes_cr']))

education_cr_df['novelty_impact_rate'] = education_cr_df.apply(calculateCRNoveltyImpact, axis=1)
health_cr_df['novelty_impact_rate'] = health_cr_df.apply(calculateCRNoveltyImpact, axis=1)
culture_cr_df['novelty_impact_rate'] = culture_cr_df.apply(calculateCRNoveltyImpact, axis=1)
public_space_cr_df['novelty_impact_rate'] = public_space_cr_df.apply(calculateCRNoveltyImpact, axis=1)
welfare_cr_df['novelty_impact_rate'] = welfare_cr_df.apply(calculateCRNoveltyImpact, axis=1)
sport_cr_df['novelty_impact_rate'] = sport_cr_df.apply(calculateCRNoveltyImpact, axis=1)
public_transit_and_roads_cr_df['novelty_impact_rate'] = public_transit_and_roads_cr_df.apply(calculateCRNoveltyImpact, axis=1)
urban_greenery_cr_df['novelty_impact_rate'] = urban_greenery_cr_df.apply(calculateCRNoveltyImpact, axis=1)
env_protection_cr_df['novelty_impact_rate'] = env_protection_cr_df.apply(calculateCRNoveltyImpact, axis=1)


In [ ]:
## introducing impact metric value from novelty of exclusiveness for project representation
def calculatePRNoveltyImpact(row):
    if row['ug_pr'] == 0:
        return (0 - (1 * row['excl_mes_pr'] / row['mes_pr']))
    elif row['mes_pr'] == 0:
        return(1 * row['excl_ug_pr'] / row['ug_pr'])
    else:
        return ((1 * row['excl_ug_pr'] / row['ug_pr']) - (1 * row['excl_mes_pr'] / row['mes_pr']))

education_pr_df['novelty_impact_rate'] = education_pr_df.apply(calculatePRNoveltyImpact, axis=1)
health_pr_df['novelty_impact_rate'] = health_pr_df.apply(calculatePRNoveltyImpact, axis=1)
culture_pr_df['novelty_impact_rate'] = culture_pr_df.apply(calculatePRNoveltyImpact, axis=1)
public_space_pr_df['novelty_impact_rate'] = public_space_pr_df.apply(calculatePRNoveltyImpact, axis=1)
welfare_pr_df['novelty_impact_rate'] = welfare_pr_df.apply(calculatePRNoveltyImpact, axis=1)
sport_pr_df['novelty_impact_rate'] = sport_pr_df.apply(calculatePRNoveltyImpact, axis=1)
public_transit_and_roads_pr_df['novelty_impact_rate'] = public_transit_and_roads_pr_df.apply(calculatePRNoveltyImpact, axis=1)
urban_greenery_pr_df['novelty_impact_rate'] = urban_greenery_pr_df.apply(calculatePRNoveltyImpact, axis=1)
env_protection_pr_df['novelty_impact_rate'] = env_protection_pr_df.apply(calculatePRNoveltyImpact, axis=1)

### Plot for the 9 x 4 line plot showing 4 metrics across 9 categories, each plot has two lines - impact line and corresponding novelty due to impact line

In [ ]:
# Plotting 9 x 4 plots; for each of the 9 categories, 4 different plots
fig, axes = plt.subplots(9, 4, figsize=(16, 36), sharex=True, sharey=True)

## for Education
education_bs_plot_df = education_bs_df[['election_id', 'diff_greedy_mes_relative_winners_cost_pct_education', 'diff_go_mo_relative_winners_cost_pct_education', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_cost_pct_education', ascending=False).reset_index()
education_wr_plot_df = education_wr_df[['election_id', 'diff_greedy_mes_relative_winners_count_pct_education', 'diff_go_mo_relative_winners_count_pct_education', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_count_pct_education', ascending=False).reset_index()
education_cr_plot_df = education_cr_df[['election_id', 'diff_greedy_mes_relative_proposals_cost_pct_education', 'diff_go_mo_relative_proposals_cost_pct_education', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_cost_pct_education', ascending=False).reset_index()
education_pr_plot_df = education_pr_df[['election_id', 'diff_greedy_mes_relative_proposals_count_pct_education', 'diff_go_mo_relative_proposals_count_pct_education', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_count_pct_education', ascending=False).reset_index()

axes[0][0].axhline(y=0, color='black', alpha=0.4)
axes[0][0].scatter(range(len(education_bs_plot_df)), education_bs_plot_df['diff_greedy_mes_relative_winners_cost_pct_education'],  marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[0][0].scatter(range(len(education_bs_plot_df)), education_bs_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[0][0].plot(range(len(education_bs_plot_df)), education_bs_plot_df['novelty_impact_rate'], color='red')
axes[0][0].scatter(range(len(education_bs_plot_df)), education_bs_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[0][0].plot(range(len(education_bs_plot_df)), education_bs_plot_df['within_novelty'], color='pink')
axes[0][0].scatter(range(len(education_bs_plot_df)), education_bs_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[0][0].plot(range(len(education_bs_plot_df)), education_bs_plot_df['between_novelty'], color='green')
axes[0][0].set_title('Budget Share', fontsize=16)
axes[0][0].set_ylabel('Education', fontsize=16)

axes[0][1].axhline(y=0, color='black', alpha=0.4)
axes[0][1].scatter(range(len(education_wr_plot_df)), education_wr_plot_df['diff_greedy_mes_relative_winners_count_pct_education'], marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[0][1].scatter(range(len(education_wr_plot_df)), education_wr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[0][1].plot(range(len(education_wr_plot_df)), education_wr_plot_df['novelty_impact_rate'], color='red')
axes[0][1].scatter(range(len(education_wr_plot_df)), education_wr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[0][1].plot(range(len(education_wr_plot_df)), education_wr_plot_df['within_novelty'], color='pink')
axes[0][1].scatter(range(len(education_wr_plot_df)), education_wr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[0][1].plot(range(len(education_wr_plot_df)), education_wr_plot_df['between_novelty'], color='green')

axes[0][1].set_title('Winning Rate', fontsize=16)
axes[0][2].axhline(y=0, color='black', alpha=0.4)
axes[0][2].scatter(range(len(education_cr_plot_df)), education_cr_plot_df['diff_greedy_mes_relative_proposals_cost_pct_education'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[0][2].scatter(range(len(education_cr_plot_df)), education_cr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[0][2].plot(range(len(education_cr_plot_df)), education_cr_plot_df['novelty_impact_rate'], color='red')
axes[0][2].scatter(range(len(education_cr_plot_df)), education_cr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[0][2].plot(range(len(education_cr_plot_df)), education_cr_plot_df['within_novelty'], color='pink')
axes[0][2].scatter(range(len(education_cr_plot_df)), education_cr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[0][2].plot(range(len(education_cr_plot_df)), education_cr_plot_df['between_novelty'], color='green')
axes[0][2].set_title('Cost Representation', fontsize=16)

axes[0][3].axhline(y=0, color='black', alpha=0.4)
axes[0][3].scatter(range(len(education_pr_plot_df)), education_pr_plot_df['diff_greedy_mes_relative_proposals_count_pct_education'], label='Impact Loss', marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[0][3].scatter(range(len(education_pr_plot_df)), education_pr_plot_df['novelty_impact_rate'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='red')
axes[0][3].plot(range(len(education_pr_plot_df)), education_pr_plot_df['novelty_impact_rate'], color='red')
axes[0][3].scatter(range(len(education_pr_plot_df)), education_pr_plot_df['within_novelty'], label='In-Novelty', marker='o', facecolors='none', alpha=0.2, color='pink')
axes[0][3].plot(range(len(education_pr_plot_df)), education_pr_plot_df['within_novelty'], color='pink')
axes[0][3].scatter(range(len(education_pr_plot_df)), education_pr_plot_df['between_novelty'], label='Between-Novelty', marker='o', facecolors='none', alpha=0.2, color='green')
axes[0][3].plot(range(len(education_pr_plot_df)), education_pr_plot_df['between_novelty'], color='green')
axes[0][3].legend()
axes[0][3].set_title('Project Representation', fontsize=16)

## for Culture
culture_bs_plot_df = culture_bs_df[['election_id', 'diff_greedy_mes_relative_winners_cost_pct_culture', 'diff_go_mo_relative_winners_cost_pct_culture', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_cost_pct_culture', ascending=False).reset_index()
culture_wr_plot_df = culture_wr_df[['election_id', 'diff_greedy_mes_relative_winners_count_pct_culture', 'diff_go_mo_relative_winners_count_pct_culture', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_count_pct_culture', ascending=False).reset_index()
culture_cr_plot_df = culture_cr_df[['election_id', 'diff_greedy_mes_relative_proposals_cost_pct_culture', 'diff_go_mo_relative_proposals_cost_pct_culture', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_cost_pct_culture', ascending=False).reset_index()
culture_pr_plot_df = culture_pr_df[['election_id', 'diff_greedy_mes_relative_proposals_count_pct_culture', 'diff_go_mo_relative_proposals_count_pct_culture', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_count_pct_culture', ascending=False).reset_index()

axes[1][0].axhline(y=0, color='black', alpha=0.4)
axes[1][0].scatter(range(len(culture_bs_plot_df)), culture_bs_plot_df['diff_greedy_mes_relative_winners_cost_pct_culture'], label='Loss', marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[1][0].scatter(range(len(culture_bs_plot_df)), culture_bs_plot_df['novelty_impact_rate'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='red')
axes[1][0].plot(range(len(culture_bs_plot_df)), culture_bs_plot_df['novelty_impact_rate'], color='red')
axes[1][0].scatter(range(len(culture_bs_plot_df)), culture_bs_plot_df['within_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='pink')
axes[1][0].plot(range(len(culture_bs_plot_df)), culture_bs_plot_df['within_novelty'], color='pink')
axes[1][0].scatter(range(len(culture_bs_plot_df)), culture_bs_plot_df['between_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='green')
axes[1][0].plot(range(len(culture_bs_plot_df)), culture_bs_plot_df['between_novelty'], color='green')
axes[1][0].set_ylabel('Culture', fontsize=16)

axes[1][1].axhline(y=0, color='black', alpha=0.4)
axes[1][1].scatter(range(len(culture_wr_plot_df)), culture_wr_plot_df['diff_greedy_mes_relative_winners_count_pct_culture'], marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[1][1].scatter(range(len(culture_wr_plot_df)), culture_wr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[1][1].plot(range(len(culture_wr_plot_df)), culture_wr_plot_df['novelty_impact_rate'], color='red')
axes[1][1].scatter(range(len(culture_wr_plot_df)), culture_wr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[1][1].plot(range(len(culture_wr_plot_df)), culture_wr_plot_df['within_novelty'], color='pink')
axes[1][1].scatter(range(len(culture_wr_plot_df)), culture_wr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[1][1].plot(range(len(culture_wr_plot_df)), culture_wr_plot_df['between_novelty'], color='green')

axes[1][2].axhline(y=0, color='black', alpha=0.4)
axes[1][2].scatter(range(len(culture_cr_plot_df)), culture_cr_plot_df['diff_greedy_mes_relative_proposals_cost_pct_culture'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[1][2].scatter(range(len(culture_cr_plot_df)), culture_cr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[1][2].plot(range(len(culture_cr_plot_df)), culture_cr_plot_df['novelty_impact_rate'], color='red')
axes[1][2].scatter(range(len(culture_cr_plot_df)), culture_cr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[1][2].plot(range(len(culture_cr_plot_df)), culture_cr_plot_df['within_novelty'], color='pink')
axes[1][2].scatter(range(len(culture_cr_plot_df)), culture_cr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[1][2].plot(range(len(culture_cr_plot_df)), culture_cr_plot_df['between_novelty'], color='green')

axes[1][3].axhline(y=0, color='black', alpha=0.4)
axes[1][3].scatter(range(len(culture_pr_plot_df)), culture_pr_plot_df['diff_greedy_mes_relative_proposals_count_pct_culture'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[1][3].scatter(range(len(culture_pr_plot_df)), culture_pr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[1][3].plot(range(len(culture_pr_plot_df)), culture_pr_plot_df['novelty_impact_rate'], color='red')
axes[1][3].scatter(range(len(culture_pr_plot_df)), culture_pr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[1][3].plot(range(len(culture_pr_plot_df)), culture_pr_plot_df['within_novelty'], color='pink')
axes[1][3].scatter(range(len(culture_pr_plot_df)), culture_pr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[1][3].plot(range(len(culture_pr_plot_df)), culture_pr_plot_df['between_novelty'], color='green')

## for Welfare
welfare_bs_plot_df = welfare_bs_df[['election_id', 'diff_greedy_mes_relative_winners_cost_pct_welfare', 'diff_go_mo_relative_winners_cost_pct_welfare', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_cost_pct_welfare', ascending=False).reset_index()
welfare_wr_plot_df = welfare_wr_df[['election_id', 'diff_greedy_mes_relative_winners_count_pct_welfare', 'diff_go_mo_relative_winners_count_pct_welfare', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_count_pct_welfare', ascending=False).reset_index()
welfare_cr_plot_df = welfare_cr_df[['election_id', 'diff_greedy_mes_relative_proposals_cost_pct_welfare', 'diff_go_mo_relative_proposals_cost_pct_welfare', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_cost_pct_welfare', ascending=False).reset_index()
welfare_pr_plot_df = welfare_pr_df[['election_id', 'diff_greedy_mes_relative_proposals_count_pct_welfare', 'diff_go_mo_relative_proposals_count_pct_welfare', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_count_pct_welfare', ascending=False).reset_index()

axes[2][0].axhline(y=0, color='black', alpha=0.4)
axes[2][0].scatter(range(len(welfare_bs_plot_df)), welfare_bs_plot_df['diff_greedy_mes_relative_winners_cost_pct_welfare'], label='Loss', marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[2][0].scatter(range(len(welfare_bs_plot_df)), welfare_bs_plot_df['novelty_impact_rate'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='red')
axes[2][0].plot(range(len(welfare_bs_plot_df)), welfare_bs_plot_df['novelty_impact_rate'], color='red')
axes[2][0].scatter(range(len(welfare_bs_plot_df)), welfare_bs_plot_df['within_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='pink')
axes[2][0].plot(range(len(welfare_bs_plot_df)), welfare_bs_plot_df['within_novelty'], color='pink')
axes[2][0].scatter(range(len(welfare_bs_plot_df)), welfare_bs_plot_df['between_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='green')
axes[2][0].plot(range(len(welfare_bs_plot_df)), welfare_bs_plot_df['between_novelty'], color='green')
axes[2][0].set_ylabel('Welfare', fontsize=16)

axes[2][1].axhline(y=0, color='black', alpha=0.4)
axes[2][1].scatter(range(len(welfare_wr_plot_df)), welfare_wr_plot_df['diff_greedy_mes_relative_winners_count_pct_welfare'], marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[2][1].scatter(range(len(welfare_wr_plot_df)), welfare_wr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[2][1].plot(range(len(welfare_wr_plot_df)), welfare_wr_plot_df['novelty_impact_rate'], color='red')
axes[2][1].scatter(range(len(welfare_wr_plot_df)), welfare_wr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[2][1].plot(range(len(welfare_wr_plot_df)), welfare_wr_plot_df['within_novelty'], color='pink')
axes[2][1].scatter(range(len(welfare_wr_plot_df)), welfare_wr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[2][1].plot(range(len(welfare_wr_plot_df)), welfare_wr_plot_df['between_novelty'], color='green')

axes[2][2].axhline(y=0, color='black', alpha=0.4)
axes[2][2].scatter(range(len(welfare_cr_plot_df)), welfare_cr_plot_df['diff_greedy_mes_relative_proposals_cost_pct_welfare'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[2][2].scatter(range(len(welfare_cr_plot_df)), welfare_cr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[2][2].plot(range(len(welfare_cr_plot_df)), welfare_cr_plot_df['novelty_impact_rate'], color='red')
axes[2][2].scatter(range(len(welfare_cr_plot_df)), welfare_cr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[2][2].plot(range(len(welfare_cr_plot_df)), welfare_cr_plot_df['within_novelty'], color='pink')
axes[2][2].scatter(range(len(welfare_cr_plot_df)), welfare_cr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[2][2].plot(range(len(welfare_cr_plot_df)), welfare_cr_plot_df['between_novelty'], color='green')

axes[2][3].axhline(y=0, color='black', alpha=0.4)
axes[2][3].scatter(range(len(welfare_pr_plot_df)), welfare_pr_plot_df['diff_greedy_mes_relative_proposals_count_pct_welfare'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[2][3].scatter(range(len(welfare_pr_plot_df)), welfare_pr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[2][3].plot(range(len(welfare_pr_plot_df)), welfare_pr_plot_df['novelty_impact_rate'], color='red')
axes[2][3].scatter(range(len(welfare_pr_plot_df)), welfare_pr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[2][3].plot(range(len(welfare_pr_plot_df)), welfare_pr_plot_df['within_novelty'], color='pink')
axes[2][3].scatter(range(len(welfare_pr_plot_df)), welfare_pr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[2][3].plot(range(len(welfare_pr_plot_df)), welfare_pr_plot_df['between_novelty'], color='green')

## for Health
health_bs_plot_df = health_bs_df[['election_id', 'diff_greedy_mes_relative_winners_cost_pct_health', 'diff_go_mo_relative_winners_cost_pct_health', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_cost_pct_health', ascending=False).reset_index()
health_wr_plot_df = health_wr_df[['election_id', 'diff_greedy_mes_relative_winners_count_pct_health', 'diff_go_mo_relative_winners_count_pct_health', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_count_pct_health', ascending=False).reset_index()
health_cr_plot_df = health_cr_df[['election_id', 'diff_greedy_mes_relative_proposals_cost_pct_health', 'diff_go_mo_relative_proposals_cost_pct_health', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_cost_pct_health', ascending=False).reset_index()
health_pr_plot_df = health_pr_df[['election_id', 'diff_greedy_mes_relative_proposals_count_pct_health', 'diff_go_mo_relative_proposals_count_pct_health', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_count_pct_health', ascending=False).reset_index()

axes[3][0].axhline(y=0, color='black', alpha=0.4)
axes[3][0].scatter(range(len(health_bs_plot_df)), health_bs_plot_df['diff_greedy_mes_relative_winners_cost_pct_health'], label='Loss', marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[3][0].scatter(range(len(health_bs_plot_df)), health_bs_plot_df['novelty_impact_rate'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='red')
axes[3][0].plot(range(len(health_bs_plot_df)), health_bs_plot_df['novelty_impact_rate'], color='red')
axes[3][0].scatter(range(len(health_bs_plot_df)), health_bs_plot_df['within_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='pink')
axes[3][0].plot(range(len(health_bs_plot_df)), health_bs_plot_df['within_novelty'], color='pink')
axes[3][0].scatter(range(len(health_bs_plot_df)), health_bs_plot_df['between_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='green')
axes[3][0].plot(range(len(health_bs_plot_df)), health_bs_plot_df['between_novelty'], color='green')
axes[3][0].set_ylabel('Health', fontsize=16)

axes[3][1].axhline(y=0, color='black', alpha=0.4)
axes[3][1].scatter(range(len(health_wr_plot_df)), health_wr_plot_df['diff_greedy_mes_relative_winners_count_pct_health'], marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[3][1].scatter(range(len(health_wr_plot_df)), health_wr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[3][1].plot(range(len(health_wr_plot_df)), health_wr_plot_df['novelty_impact_rate'], color='red')
axes[3][1].scatter(range(len(health_wr_plot_df)), health_wr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[3][1].plot(range(len(health_wr_plot_df)), health_wr_plot_df['within_novelty'], color='pink')
axes[3][1].scatter(range(len(health_wr_plot_df)), health_wr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[3][1].plot(range(len(health_wr_plot_df)), health_wr_plot_df['between_novelty'], color='green')

axes[3][2].axhline(y=0, color='black', alpha=0.4)
axes[3][2].scatter(range(len(health_cr_plot_df)), health_cr_plot_df['diff_greedy_mes_relative_proposals_cost_pct_health'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[3][2].scatter(range(len(health_cr_plot_df)), health_cr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[3][2].plot(range(len(health_cr_plot_df)), health_cr_plot_df['novelty_impact_rate'], color='red')
axes[3][2].scatter(range(len(health_cr_plot_df)), health_cr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[3][2].plot(range(len(health_cr_plot_df)), health_cr_plot_df['within_novelty'], color='pink')
axes[3][2].scatter(range(len(health_cr_plot_df)), health_cr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[3][2].plot(range(len(health_cr_plot_df)), health_cr_plot_df['between_novelty'], color='green')

axes[3][3].axhline(y=0, color='black', alpha=0.4)
axes[3][3].scatter(range(len(health_pr_plot_df)), health_pr_plot_df['diff_greedy_mes_relative_proposals_count_pct_health'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[3][3].scatter(range(len(health_pr_plot_df)), health_pr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[3][3].plot(range(len(health_pr_plot_df)), health_pr_plot_df['novelty_impact_rate'], color='red')
axes[3][3].scatter(range(len(health_pr_plot_df)), health_pr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[3][3].plot(range(len(health_pr_plot_df)), health_pr_plot_df['within_novelty'], color='pink')
axes[3][3].scatter(range(len(health_pr_plot_df)), health_pr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[3][3].plot(range(len(health_pr_plot_df)), health_pr_plot_df['between_novelty'], color='green')

## for Public Space
public_space_bs_plot_df = public_space_bs_df[['election_id', 'diff_greedy_mes_relative_winners_cost_pct_public_space', 'diff_go_mo_relative_winners_cost_pct_public_space', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_cost_pct_public_space', ascending=False).reset_index()
public_space_wr_plot_df = public_space_wr_df[['election_id', 'diff_greedy_mes_relative_winners_count_pct_public_space', 'diff_go_mo_relative_winners_count_pct_public_space', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_count_pct_public_space', ascending=False).reset_index()
public_space_cr_plot_df = public_space_cr_df[['election_id', 'diff_greedy_mes_relative_proposals_cost_pct_public_space', 'diff_go_mo_relative_proposals_cost_pct_public_space', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_cost_pct_public_space', ascending=False).reset_index()
public_space_pr_plot_df = public_space_pr_df[['election_id', 'diff_greedy_mes_relative_proposals_count_pct_public_space', 'diff_go_mo_relative_proposals_count_pct_public_space', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_count_pct_public_space', ascending=False).reset_index()

axes[4][0].axhline(y=0, color='black', alpha=0.4)
axes[4][0].scatter(range(len(public_space_bs_plot_df)), public_space_bs_plot_df['diff_greedy_mes_relative_winners_cost_pct_public_space'], label='Loss', marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[4][0].scatter(range(len(public_space_bs_plot_df)), public_space_bs_plot_df['novelty_impact_rate'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='red')
axes[4][0].plot(range(len(public_space_bs_plot_df)), public_space_bs_plot_df['novelty_impact_rate'], color='red')
axes[4][0].scatter(range(len(public_space_bs_plot_df)), public_space_bs_plot_df['within_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='pink')
axes[4][0].plot(range(len(public_space_bs_plot_df)), public_space_bs_plot_df['within_novelty'], color='pink')
axes[4][0].scatter(range(len(public_space_bs_plot_df)), public_space_bs_plot_df['between_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='green')
axes[4][0].plot(range(len(public_space_bs_plot_df)), public_space_bs_plot_df['between_novelty'], color='green')
axes[4][0].set_ylabel('Public Space', fontsize=16)

axes[4][1].axhline(y=0, color='black', alpha=0.4)
axes[4][1].scatter(range(len(public_space_wr_plot_df)), public_space_wr_plot_df['diff_greedy_mes_relative_winners_count_pct_public_space'], marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[4][1].scatter(range(len(public_space_wr_plot_df)), public_space_wr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[4][1].plot(range(len(public_space_wr_plot_df)), public_space_wr_plot_df['novelty_impact_rate'], color='red')
axes[4][1].scatter(range(len(public_space_wr_plot_df)), public_space_wr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[4][1].plot(range(len(public_space_wr_plot_df)), public_space_wr_plot_df['within_novelty'], color='pink')
axes[4][1].scatter(range(len(public_space_wr_plot_df)), public_space_wr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[4][1].plot(range(len(public_space_wr_plot_df)), public_space_wr_plot_df['between_novelty'], color='green')

axes[4][2].axhline(y=0, color='black', alpha=0.4)
axes[4][2].scatter(range(len(public_space_cr_plot_df)), public_space_cr_plot_df['diff_greedy_mes_relative_proposals_cost_pct_public_space'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[4][2].scatter(range(len(public_space_cr_plot_df)), public_space_cr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[4][2].plot(range(len(public_space_cr_plot_df)), public_space_cr_plot_df['novelty_impact_rate'], color='red')
axes[4][2].scatter(range(len(public_space_cr_plot_df)), public_space_cr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[4][2].plot(range(len(public_space_cr_plot_df)), public_space_cr_plot_df['within_novelty'], color='pink')
axes[4][2].scatter(range(len(public_space_cr_plot_df)), public_space_cr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[4][2].plot(range(len(public_space_cr_plot_df)), public_space_cr_plot_df['between_novelty'], color='green')

axes[4][3].axhline(y=0, color='black', alpha=0.4)
axes[4][3].scatter(range(len(public_space_pr_plot_df)), public_space_pr_plot_df['diff_greedy_mes_relative_proposals_count_pct_public_space'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[4][3].scatter(range(len(public_space_pr_plot_df)), public_space_pr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[4][3].plot(range(len(public_space_pr_plot_df)), public_space_pr_plot_df['novelty_impact_rate'], color='red')
axes[4][3].scatter(range(len(public_space_pr_plot_df)), public_space_pr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[4][3].plot(range(len(public_space_pr_plot_df)), public_space_pr_plot_df['within_novelty'], color='pink')
axes[4][3].scatter(range(len(public_space_pr_plot_df)), public_space_pr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[4][3].plot(range(len(public_space_pr_plot_df)), public_space_pr_plot_df['between_novelty'], color='green')

## for Urban Greenery
urban_greenery_bs_plot_df = urban_greenery_bs_df[['election_id', 'diff_greedy_mes_relative_winners_cost_pct_urban_greenery', 'diff_go_mo_relative_winners_cost_pct_urban_greenery', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_cost_pct_urban_greenery', ascending=False).reset_index()
urban_greenery_wr_plot_df = urban_greenery_wr_df[['election_id', 'diff_greedy_mes_relative_winners_count_pct_urban_greenery', 'diff_go_mo_relative_winners_count_pct_urban_greenery', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_count_pct_urban_greenery', ascending=False).reset_index()
urban_greenery_cr_plot_df = urban_greenery_cr_df[['election_id', 'diff_greedy_mes_relative_proposals_cost_pct_urban_greenery', 'diff_go_mo_relative_proposals_cost_pct_urban_greenery', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_cost_pct_urban_greenery', ascending=False).reset_index()
urban_greenery_pr_plot_df = urban_greenery_pr_df[['election_id', 'diff_greedy_mes_relative_proposals_count_pct_urban_greenery', 'diff_go_mo_relative_proposals_count_pct_urban_greenery', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_count_pct_urban_greenery', ascending=False).reset_index()

axes[5][0].axhline(y=0, color='black', alpha=0.4)
axes[5][0].scatter(range(len(urban_greenery_bs_plot_df)), urban_greenery_bs_plot_df['diff_greedy_mes_relative_winners_cost_pct_urban_greenery'], label='Loss', marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[5][0].scatter(range(len(urban_greenery_bs_plot_df)), urban_greenery_bs_plot_df['novelty_impact_rate'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='red')
axes[5][0].plot(range(len(urban_greenery_bs_plot_df)), urban_greenery_bs_plot_df['novelty_impact_rate'], color='red')
axes[5][0].scatter(range(len(urban_greenery_bs_plot_df)), urban_greenery_bs_plot_df['within_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='pink')
axes[5][0].plot(range(len(urban_greenery_bs_plot_df)), urban_greenery_bs_plot_df['within_novelty'], color='pink')
axes[5][0].scatter(range(len(urban_greenery_bs_plot_df)), urban_greenery_bs_plot_df['between_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='green')
axes[5][0].plot(range(len(urban_greenery_bs_plot_df)), urban_greenery_bs_plot_df['between_novelty'], color='green')
axes[5][0].set_ylabel('Urban Greenery', fontsize=16)

axes[5][1].axhline(y=0, color='black', alpha=0.4)
axes[5][1].scatter(range(len(urban_greenery_wr_plot_df)), urban_greenery_wr_plot_df['diff_greedy_mes_relative_winners_count_pct_urban_greenery'], marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[5][1].scatter(range(len(urban_greenery_wr_plot_df)), urban_greenery_wr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[5][1].plot(range(len(urban_greenery_wr_plot_df)), urban_greenery_wr_plot_df['novelty_impact_rate'], color='red')
axes[5][1].scatter(range(len(urban_greenery_wr_plot_df)), urban_greenery_wr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[5][1].plot(range(len(urban_greenery_wr_plot_df)), urban_greenery_wr_plot_df['within_novelty'], color='pink')
axes[5][1].scatter(range(len(urban_greenery_wr_plot_df)), urban_greenery_wr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[5][1].plot(range(len(urban_greenery_wr_plot_df)), urban_greenery_wr_plot_df['between_novelty'], color='green')

axes[5][2].axhline(y=0, color='black', alpha=0.4)
axes[5][2].scatter(range(len(urban_greenery_cr_plot_df)), urban_greenery_cr_plot_df['diff_greedy_mes_relative_proposals_cost_pct_urban_greenery'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[5][2].scatter(range(len(urban_greenery_cr_plot_df)), urban_greenery_cr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[5][2].plot(range(len(urban_greenery_cr_plot_df)), urban_greenery_cr_plot_df['novelty_impact_rate'], color='red')
axes[5][2].scatter(range(len(urban_greenery_cr_plot_df)), urban_greenery_cr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[5][2].plot(range(len(urban_greenery_cr_plot_df)), urban_greenery_cr_plot_df['within_novelty'], color='pink')
axes[5][2].scatter(range(len(urban_greenery_cr_plot_df)), urban_greenery_cr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[5][2].plot(range(len(urban_greenery_cr_plot_df)), urban_greenery_cr_plot_df['between_novelty'], color='green')

axes[5][3].axhline(y=0, color='black', alpha=0.4)
axes[5][3].scatter(range(len(urban_greenery_pr_plot_df)), urban_greenery_pr_plot_df['diff_greedy_mes_relative_proposals_count_pct_urban_greenery'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[5][3].scatter(range(len(urban_greenery_pr_plot_df)), urban_greenery_pr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[5][3].plot(range(len(urban_greenery_pr_plot_df)), urban_greenery_pr_plot_df['novelty_impact_rate'], color='red')
axes[5][3].scatter(range(len(urban_greenery_pr_plot_df)), urban_greenery_pr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[5][3].plot(range(len(urban_greenery_pr_plot_df)), urban_greenery_pr_plot_df['within_novelty'], color='pink')
axes[5][3].scatter(range(len(urban_greenery_pr_plot_df)), urban_greenery_pr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[5][3].plot(range(len(urban_greenery_pr_plot_df)), urban_greenery_pr_plot_df['between_novelty'], color='green')

## for Public Transit
public_transit_and_roads_bs_plot_df = public_transit_and_roads_bs_df[['election_id', 'diff_greedy_mes_relative_winners_cost_pct_public_transit_and_roads', 'diff_go_mo_relative_winners_cost_pct_public_transit_and_roads', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_cost_pct_public_transit_and_roads', ascending=False).reset_index()
public_transit_and_roads_wr_plot_df = public_transit_and_roads_wr_df[['election_id', 'diff_greedy_mes_relative_winners_count_pct_public_transit_and_roads', 'diff_go_mo_relative_winners_count_pct_public_transit_and_roads', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_count_pct_public_transit_and_roads', ascending=False).reset_index()
public_transit_and_roads_cr_plot_df = public_transit_and_roads_cr_df[['election_id', 'diff_greedy_mes_relative_proposals_cost_pct_public_transit_and_roads', 'diff_go_mo_relative_proposals_cost_pct_public_transit_and_roads', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_cost_pct_public_transit_and_roads', ascending=False).reset_index()
public_transit_and_roads_pr_plot_df = public_transit_and_roads_pr_df[['election_id', 'diff_greedy_mes_relative_proposals_count_pct_public_transit_and_roads', 'diff_go_mo_relative_proposals_count_pct_public_transit_and_roads', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_count_pct_public_transit_and_roads', ascending=False).reset_index()

axes[6][0].axhline(y=0, color='black', alpha=0.4)
axes[6][0].scatter(range(len(public_transit_and_roads_bs_plot_df)), public_transit_and_roads_bs_plot_df['diff_greedy_mes_relative_winners_cost_pct_public_transit_and_roads'], label='Loss', marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[6][0].scatter(range(len(public_transit_and_roads_bs_plot_df)), public_transit_and_roads_bs_plot_df['novelty_impact_rate'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='red')
axes[6][0].plot(range(len(public_transit_and_roads_bs_plot_df)), public_transit_and_roads_bs_plot_df['novelty_impact_rate'], color='red')
axes[6][0].scatter(range(len(public_transit_and_roads_bs_plot_df)), public_transit_and_roads_bs_plot_df['within_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='pink')
axes[6][0].plot(range(len(public_transit_and_roads_bs_plot_df)), public_transit_and_roads_bs_plot_df['within_novelty'], color='pink')
axes[6][0].scatter(range(len(public_transit_and_roads_bs_plot_df)), public_transit_and_roads_bs_plot_df['between_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='green')
axes[6][0].plot(range(len(public_transit_and_roads_bs_plot_df)), public_transit_and_roads_bs_plot_df['between_novelty'], color='green')
axes[6][0].set_ylabel('Public Transit', fontsize=16)

axes[6][1].axhline(y=0, color='black', alpha=0.4)
axes[6][1].scatter(range(len(public_transit_and_roads_wr_plot_df)), public_transit_and_roads_wr_plot_df['diff_greedy_mes_relative_winners_count_pct_public_transit_and_roads'], marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[6][1].scatter(range(len(public_transit_and_roads_wr_plot_df)), public_transit_and_roads_wr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[6][1].plot(range(len(public_transit_and_roads_wr_plot_df)), public_transit_and_roads_wr_plot_df['novelty_impact_rate'], color='red')
axes[6][1].scatter(range(len(public_transit_and_roads_wr_plot_df)), public_transit_and_roads_wr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[6][1].plot(range(len(public_transit_and_roads_wr_plot_df)), public_transit_and_roads_wr_plot_df['within_novelty'], color='pink')
axes[6][1].scatter(range(len(public_transit_and_roads_wr_plot_df)), public_transit_and_roads_wr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[6][1].plot(range(len(public_transit_and_roads_wr_plot_df)), public_transit_and_roads_wr_plot_df['between_novelty'], color='green')

axes[6][2].axhline(y=0, color='black', alpha=0.4)
axes[6][2].scatter(range(len(public_transit_and_roads_cr_plot_df)), public_transit_and_roads_cr_plot_df['diff_greedy_mes_relative_proposals_cost_pct_public_transit_and_roads'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[6][2].scatter(range(len(public_transit_and_roads_cr_plot_df)), public_transit_and_roads_cr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[6][2].plot(range(len(public_transit_and_roads_cr_plot_df)), public_transit_and_roads_cr_plot_df['novelty_impact_rate'], color='red')
axes[6][2].scatter(range(len(public_transit_and_roads_cr_plot_df)), public_transit_and_roads_cr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[6][2].plot(range(len(public_transit_and_roads_cr_plot_df)), public_transit_and_roads_cr_plot_df['within_novelty'], color='pink')
axes[6][2].scatter(range(len(public_transit_and_roads_cr_plot_df)), public_transit_and_roads_cr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[6][2].plot(range(len(public_transit_and_roads_cr_plot_df)), public_transit_and_roads_cr_plot_df['between_novelty'], color='green')

axes[6][3].axhline(y=0, color='black', alpha=0.4)
axes[6][3].scatter(range(len(public_transit_and_roads_pr_plot_df)), public_transit_and_roads_pr_plot_df['diff_greedy_mes_relative_proposals_count_pct_public_transit_and_roads'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[6][3].scatter(range(len(public_transit_and_roads_pr_plot_df)), public_transit_and_roads_pr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[6][3].plot(range(len(public_transit_and_roads_pr_plot_df)), public_transit_and_roads_pr_plot_df['novelty_impact_rate'], color='red')
axes[6][3].scatter(range(len(public_transit_and_roads_pr_plot_df)), public_transit_and_roads_pr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[6][3].plot(range(len(public_transit_and_roads_pr_plot_df)), public_transit_and_roads_pr_plot_df['within_novelty'], color='pink')
axes[6][3].scatter(range(len(public_transit_and_roads_pr_plot_df)), public_transit_and_roads_pr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[6][3].plot(range(len(public_transit_and_roads_pr_plot_df)), public_transit_and_roads_pr_plot_df['between_novelty'], color='green')

## for Env Protection
env_protection_bs_plot_df = env_protection_bs_df[['election_id', 'diff_greedy_mes_relative_winners_cost_pct_env_protection', 'diff_go_mo_relative_winners_cost_pct_env_protection', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_cost_pct_env_protection', ascending=False).reset_index()
env_protection_wr_plot_df = env_protection_wr_df[['election_id', 'diff_greedy_mes_relative_winners_count_pct_env_protection', 'diff_go_mo_relative_winners_count_pct_env_protection', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_count_pct_env_protection', ascending=False).reset_index()
env_protection_cr_plot_df = env_protection_cr_df[['election_id', 'diff_greedy_mes_relative_proposals_cost_pct_env_protection', 'diff_go_mo_relative_proposals_cost_pct_env_protection', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_cost_pct_env_protection', ascending=False).reset_index()
env_protection_pr_plot_df = env_protection_pr_df[['election_id', 'diff_greedy_mes_relative_proposals_count_pct_env_protection', 'diff_go_mo_relative_proposals_count_pct_env_protection', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_count_pct_env_protection', ascending=False).reset_index()

axes[7][0].axhline(y=0, color='black', alpha=0.4)
axes[7][0].scatter(range(len(env_protection_bs_plot_df)), env_protection_bs_plot_df['diff_greedy_mes_relative_winners_cost_pct_env_protection'], label='Loss', marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[7][0].scatter(range(len(env_protection_bs_plot_df)), env_protection_bs_plot_df['novelty_impact_rate'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='red')
axes[7][0].plot(range(len(env_protection_bs_plot_df)), env_protection_bs_plot_df['novelty_impact_rate'], color='red')
axes[7][0].scatter(range(len(env_protection_bs_plot_df)), env_protection_bs_plot_df['within_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='pink')
axes[7][0].plot(range(len(env_protection_bs_plot_df)), env_protection_bs_plot_df['within_novelty'], color='pink')
axes[7][0].scatter(range(len(env_protection_bs_plot_df)), env_protection_bs_plot_df['between_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='green')
axes[7][0].plot(range(len(env_protection_bs_plot_df)), env_protection_bs_plot_df['between_novelty'], color='green')
axes[7][0].set_ylabel('Env. Protection', fontsize=16)

axes[7][1].axhline(y=0, color='black', alpha=0.4)
axes[7][1].scatter(range(len(env_protection_wr_plot_df)), env_protection_wr_plot_df['diff_greedy_mes_relative_winners_count_pct_env_protection'], marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[7][1].scatter(range(len(env_protection_wr_plot_df)), env_protection_wr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[7][1].plot(range(len(env_protection_wr_plot_df)), env_protection_wr_plot_df['novelty_impact_rate'], color='red')
axes[7][1].scatter(range(len(env_protection_wr_plot_df)), env_protection_wr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[7][1].plot(range(len(env_protection_wr_plot_df)), env_protection_wr_plot_df['within_novelty'], color='pink')
axes[7][1].scatter(range(len(env_protection_wr_plot_df)), env_protection_wr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[7][1].plot(range(len(env_protection_wr_plot_df)), env_protection_wr_plot_df['between_novelty'], color='green')

axes[7][2].axhline(y=0, color='black', alpha=0.4)
axes[7][2].scatter(range(len(env_protection_cr_plot_df)), env_protection_cr_plot_df['diff_greedy_mes_relative_proposals_cost_pct_env_protection'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[7][2].scatter(range(len(env_protection_cr_plot_df)), env_protection_cr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[7][2].plot(range(len(env_protection_cr_plot_df)), env_protection_cr_plot_df['novelty_impact_rate'], color='red')
axes[7][2].scatter(range(len(env_protection_cr_plot_df)), env_protection_cr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[7][2].plot(range(len(env_protection_cr_plot_df)), env_protection_cr_plot_df['within_novelty'], color='pink')
axes[7][2].scatter(range(len(env_protection_cr_plot_df)), env_protection_cr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[7][2].plot(range(len(env_protection_cr_plot_df)), env_protection_cr_plot_df['between_novelty'], color='green')

axes[7][3].axhline(y=0, color='black', alpha=0.4)
axes[7][3].scatter(range(len(env_protection_pr_plot_df)), env_protection_pr_plot_df['diff_greedy_mes_relative_proposals_count_pct_env_protection'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[7][3].scatter(range(len(env_protection_pr_plot_df)), env_protection_pr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[7][3].plot(range(len(env_protection_pr_plot_df)), env_protection_pr_plot_df['novelty_impact_rate'], color='red')
axes[7][3].scatter(range(len(env_protection_pr_plot_df)), env_protection_pr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[7][3].plot(range(len(env_protection_pr_plot_df)), env_protection_pr_plot_df['within_novelty'], color='pink')
axes[7][3].scatter(range(len(env_protection_pr_plot_df)), env_protection_pr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[7][3].plot(range(len(env_protection_pr_plot_df)), env_protection_pr_plot_df['between_novelty'], color='green')

## for Sport
sport_bs_plot_df = sport_bs_df[['election_id', 'diff_greedy_mes_relative_winners_cost_pct_sport', 'diff_go_mo_relative_winners_cost_pct_sport', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_cost_pct_sport', ascending=False).reset_index()
sport_wr_plot_df = sport_wr_df[['election_id', 'diff_greedy_mes_relative_winners_count_pct_sport', 'diff_go_mo_relative_winners_count_pct_sport', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_winners_count_pct_sport', ascending=False).reset_index()
sport_cr_plot_df = sport_cr_df[['election_id', 'diff_greedy_mes_relative_proposals_cost_pct_sport', 'diff_go_mo_relative_proposals_cost_pct_sport', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_cost_pct_sport', ascending=False).reset_index()
sport_pr_plot_df = sport_pr_df[['election_id', 'diff_greedy_mes_relative_proposals_count_pct_sport', 'diff_go_mo_relative_proposals_count_pct_sport', 'novelty_impact_rate', 'within_novelty', 'between_novelty']].sort_values(by='diff_greedy_mes_relative_proposals_count_pct_sport', ascending=False).reset_index()

axes[8][0].axhline(y=0, color='black', alpha=0.4)
axes[8][0].scatter(range(len(sport_bs_plot_df)), sport_bs_plot_df['diff_greedy_mes_relative_winners_cost_pct_sport'], label='Loss', marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[8][0].scatter(range(len(sport_bs_plot_df)), sport_bs_plot_df['novelty_impact_rate'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='red')
axes[8][0].plot(range(len(sport_bs_plot_df)), sport_bs_plot_df['novelty_impact_rate'], color='red')
axes[8][0].scatter(range(len(sport_bs_plot_df)), sport_bs_plot_df['within_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='pink')
axes[8][0].plot(range(len(sport_bs_plot_df)), sport_bs_plot_df['within_novelty'], color='pink')
axes[8][0].scatter(range(len(sport_bs_plot_df)), sport_bs_plot_df['between_novelty'], label='Novelty', marker='o', facecolors='none', alpha=0.2, color='green')
axes[8][0].plot(range(len(sport_bs_plot_df)), sport_bs_plot_df['between_novelty'], color='green')
axes[8][0].set_ylabel('Sport', fontsize=16)

axes[8][1].axhline(y=0, color='black', alpha=0.4)
axes[8][1].scatter(range(len(sport_wr_plot_df)), sport_wr_plot_df['diff_greedy_mes_relative_winners_count_pct_sport'], marker='*', facecolors='blue', alpha=0.4, color='blue')
axes[8][1].scatter(range(len(sport_wr_plot_df)), sport_wr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[8][1].plot(range(len(sport_wr_plot_df)), sport_wr_plot_df['novelty_impact_rate'], color='red')
axes[8][1].scatter(range(len(sport_wr_plot_df)), sport_wr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[8][1].plot(range(len(sport_wr_plot_df)), sport_wr_plot_df['within_novelty'], color='pink')
axes[8][1].scatter(range(len(sport_wr_plot_df)), sport_wr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[8][1].plot(range(len(sport_wr_plot_df)), sport_wr_plot_df['between_novelty'], color='green')

axes[8][2].axhline(y=0, color='black', alpha=0.4)
axes[8][2].scatter(range(len(sport_cr_plot_df)), sport_cr_plot_df['diff_greedy_mes_relative_proposals_cost_pct_sport'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[8][2].scatter(range(len(sport_cr_plot_df)), sport_cr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[8][2].plot(range(len(sport_cr_plot_df)), sport_cr_plot_df['novelty_impact_rate'], color='red')
axes[8][2].scatter(range(len(sport_cr_plot_df)), sport_cr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[8][2].plot(range(len(sport_cr_plot_df)), sport_cr_plot_df['within_novelty'], color='pink')
axes[8][2].scatter(range(len(sport_cr_plot_df)), sport_cr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[8][2].plot(range(len(sport_cr_plot_df)), sport_cr_plot_df['between_novelty'], color='green')

axes[8][3].axhline(y=0, color='black', alpha=0.4)
axes[8][3].scatter(range(len(sport_pr_plot_df)), sport_pr_plot_df['diff_greedy_mes_relative_proposals_count_pct_sport'], marker='*', facecolor='blue', alpha=0.4, color='blue')
axes[8][3].scatter(range(len(sport_pr_plot_df)), sport_pr_plot_df['novelty_impact_rate'], marker='o', facecolors='none', alpha=0.2, color='red')
axes[8][3].plot(range(len(sport_pr_plot_df)), sport_pr_plot_df['novelty_impact_rate'], color='red')
axes[8][3].scatter(range(len(sport_pr_plot_df)), sport_pr_plot_df['within_novelty'], marker='o', facecolors='none', alpha=0.2, color='pink')
axes[8][3].plot(range(len(sport_pr_plot_df)), sport_pr_plot_df['within_novelty'], color='pink')
axes[8][3].scatter(range(len(sport_pr_plot_df)), sport_pr_plot_df['between_novelty'], marker='o', facecolors='none', alpha=0.2, color='green')
axes[8][3].plot(range(len(sport_pr_plot_df)), sport_pr_plot_df['between_novelty'], color='green')
plt.tight_layout()
plt.show()

## Segregate within and between novelty values into separate columns of loss and gain

In [ ]:
# introducing additional columns to plot into bar plots
def assignWithinNoveltyLossColumn(row):
    row['within_novelty_loss'] = abs(row['within_novelty']) if row['within_novelty'] > 0 else float('nan')
    return row['within_novelty_loss']
    
def assignWithinNoveltyGainColumn(row):
    row['within_novelty_gain'] = abs(row['within_novelty']) if row['within_novelty'] < 0 else float('nan')
    return row['within_novelty_gain']

def assignBetweenNoveltyLossColumn(row):
    row['between_novelty_loss'] = abs(row['between_novelty']) if row['between_novelty'] > 0 else float('nan')
    return row['between_novelty_loss']
    
def assignBetweenNoveltyGainColumn(row):
    row['between_novelty_gain'] = abs(row['between_novelty']) if row['between_novelty'] < 0 else float('nan')
    return row['between_novelty_gain']

within_novelty_cost_avg_map = {}
within_novelty_count_avg_map = {}
between_novelty_cost_avg_map = {}
between_novelty_count_avg_map = {}

## for education
education_bs_df['within_novelty_loss'] = education_bs_df.apply(assignWithinNoveltyLossColumn, axis=1)
education_bs_df['within_novelty_gain'] = education_bs_df.apply(assignWithinNoveltyGainColumn, axis=1)
education_bs_df['between_novelty_loss'] = education_bs_df.apply(assignBetweenNoveltyLossColumn, axis=1)
education_bs_df['between_novelty_gain'] = education_bs_df.apply(assignBetweenNoveltyGainColumn, axis=1)

education_wr_df['within_novelty_loss'] = education_wr_df.apply(assignWithinNoveltyLossColumn, axis=1)
education_wr_df['within_novelty_gain'] = education_wr_df.apply(assignWithinNoveltyGainColumn, axis=1)
education_wr_df['between_novelty_loss'] = education_wr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
education_wr_df['between_novelty_gain'] = education_wr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

education_cr_df['within_novelty_loss'] = education_cr_df.apply(assignWithinNoveltyLossColumn, axis=1)
education_cr_df['within_novelty_gain'] = education_cr_df.apply(assignWithinNoveltyGainColumn, axis=1)
education_cr_df['between_novelty_loss'] = education_cr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
education_cr_df['between_novelty_gain'] = education_cr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

education_pr_df['within_novelty_loss'] = education_pr_df.apply(assignWithinNoveltyLossColumn, axis=1)
education_pr_df['within_novelty_gain'] = education_pr_df.apply(assignWithinNoveltyGainColumn, axis=1)
education_pr_df['between_novelty_loss'] = education_pr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
education_pr_df['between_novelty_gain'] = education_pr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

within_novelty_cost_avg_map['education'] = round(education_bs_df['within_novelty'].mean(), 2)
between_novelty_cost_avg_map['education'] = round(education_bs_df['between_novelty'].mean(), 2)
within_novelty_count_avg_map['education'] = round(education_wr_df['within_novelty'].mean(), 2)
between_novelty_count_avg_map['education'] = round(education_wr_df['between_novelty'].mean(), 2)


## for culture
culture_bs_df['within_novelty_loss'] = culture_bs_df.apply(assignWithinNoveltyLossColumn, axis=1)
culture_bs_df['within_novelty_gain'] = culture_bs_df.apply(assignWithinNoveltyGainColumn, axis=1)
culture_bs_df['between_novelty_loss'] = culture_bs_df.apply(assignBetweenNoveltyLossColumn, axis=1)
culture_bs_df['between_novelty_gain'] = culture_bs_df.apply(assignBetweenNoveltyGainColumn, axis=1)

culture_wr_df['within_novelty_loss'] = culture_wr_df.apply(assignWithinNoveltyLossColumn, axis=1)
culture_wr_df['within_novelty_gain'] = culture_wr_df.apply(assignWithinNoveltyGainColumn, axis=1)
culture_wr_df['between_novelty_loss'] = culture_wr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
culture_wr_df['between_novelty_gain'] = culture_wr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

culture_cr_df['within_novelty_loss'] = culture_cr_df.apply(assignWithinNoveltyLossColumn, axis=1)
culture_cr_df['within_novelty_gain'] = culture_cr_df.apply(assignWithinNoveltyGainColumn, axis=1)
culture_cr_df['between_novelty_loss'] = culture_cr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
culture_cr_df['between_novelty_gain'] = culture_cr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

culture_pr_df['within_novelty_loss'] = culture_pr_df.apply(assignWithinNoveltyLossColumn, axis=1)
culture_pr_df['within_novelty_gain'] = culture_pr_df.apply(assignWithinNoveltyGainColumn, axis=1)
culture_pr_df['between_novelty_loss'] = culture_pr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
culture_pr_df['between_novelty_gain'] = culture_pr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

within_novelty_cost_avg_map['culture'] = round(culture_bs_df['within_novelty'].mean(), 2)
between_novelty_cost_avg_map['culture'] = round(culture_bs_df['between_novelty'].mean(), 2)
within_novelty_count_avg_map['culture'] = round(culture_wr_df['within_novelty'].mean(), 2)
between_novelty_count_avg_map['culture'] = round(culture_wr_df['between_novelty'].mean(), 2)

# ## for welfare
welfare_bs_df['within_novelty_loss'] = welfare_bs_df.apply(assignWithinNoveltyLossColumn, axis=1)
welfare_bs_df['within_novelty_gain'] = welfare_bs_df.apply(assignWithinNoveltyGainColumn, axis=1)
welfare_bs_df['between_novelty_loss'] = welfare_bs_df.apply(assignBetweenNoveltyLossColumn, axis=1)
welfare_bs_df['between_novelty_gain'] = welfare_bs_df.apply(assignBetweenNoveltyGainColumn, axis=1)

welfare_wr_df['within_novelty_loss'] = welfare_wr_df.apply(assignWithinNoveltyLossColumn, axis=1)
welfare_wr_df['within_novelty_gain'] = welfare_wr_df.apply(assignWithinNoveltyGainColumn, axis=1)
welfare_wr_df['between_novelty_loss'] = welfare_wr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
welfare_wr_df['between_novelty_gain'] = welfare_wr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

welfare_cr_df['within_novelty_loss'] = welfare_cr_df.apply(assignWithinNoveltyLossColumn, axis=1)
welfare_cr_df['within_novelty_gain'] = welfare_cr_df.apply(assignWithinNoveltyGainColumn, axis=1)
welfare_cr_df['between_novelty_loss'] = welfare_cr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
welfare_cr_df['between_novelty_gain'] = welfare_cr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

welfare_pr_df['within_novelty_loss'] = welfare_pr_df.apply(assignWithinNoveltyLossColumn, axis=1)
welfare_pr_df['within_novelty_gain'] = welfare_pr_df.apply(assignWithinNoveltyGainColumn, axis=1)
welfare_pr_df['between_novelty_loss'] = welfare_pr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
welfare_pr_df['between_novelty_gain'] = welfare_pr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

within_novelty_cost_avg_map['welfare'] = round(welfare_bs_df['within_novelty'].mean(), 2)
between_novelty_cost_avg_map['welfare'] = round(welfare_bs_df['between_novelty'].mean(), 2)
within_novelty_count_avg_map['welfare'] = round(welfare_wr_df['within_novelty'].mean(), 2)
between_novelty_count_avg_map['welfare'] = round(welfare_wr_df['between_novelty'].mean(), 2)


# ## for public space
public_space_bs_df['within_novelty_loss'] = public_space_bs_df.apply(assignWithinNoveltyLossColumn, axis=1)
public_space_bs_df['within_novelty_gain'] = public_space_bs_df.apply(assignWithinNoveltyGainColumn, axis=1)
public_space_bs_df['between_novelty_loss'] = public_space_bs_df.apply(assignBetweenNoveltyLossColumn, axis=1)
public_space_bs_df['between_novelty_gain'] = public_space_bs_df.apply(assignBetweenNoveltyGainColumn, axis=1)

public_space_wr_df['within_novelty_loss'] = public_space_wr_df.apply(assignWithinNoveltyLossColumn, axis=1)
public_space_wr_df['within_novelty_gain'] = public_space_wr_df.apply(assignWithinNoveltyGainColumn, axis=1)
public_space_wr_df['between_novelty_loss'] = public_space_wr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
public_space_wr_df['between_novelty_gain'] = public_space_wr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

public_space_cr_df['within_novelty_loss'] = public_space_cr_df.apply(assignWithinNoveltyLossColumn, axis=1)
public_space_cr_df['within_novelty_gain'] = public_space_cr_df.apply(assignWithinNoveltyGainColumn, axis=1)
public_space_cr_df['between_novelty_loss'] = public_space_cr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
public_space_cr_df['between_novelty_gain'] = public_space_cr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

public_space_pr_df['within_novelty_loss'] = public_space_pr_df.apply(assignWithinNoveltyLossColumn, axis=1)
public_space_pr_df['within_novelty_gain'] = public_space_pr_df.apply(assignWithinNoveltyGainColumn, axis=1)
public_space_pr_df['between_novelty_loss'] = public_space_pr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
public_space_pr_df['between_novelty_gain'] = public_space_pr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

within_novelty_cost_avg_map['public_space'] = round(public_space_bs_df['within_novelty'].mean(), 2)
between_novelty_cost_avg_map['public_space'] = round(public_space_bs_df['between_novelty'].mean(), 2)
within_novelty_count_avg_map['public_space'] = round(public_space_wr_df['within_novelty'].mean(), 2)
between_novelty_count_avg_map['public_space'] = round(public_space_wr_df['between_novelty'].mean(), 2)

# ## for public transit
public_transit_and_roads_bs_df['within_novelty_loss'] = public_transit_and_roads_bs_df.apply(assignWithinNoveltyLossColumn, axis=1)
public_transit_and_roads_bs_df['within_novelty_gain'] = public_transit_and_roads_bs_df.apply(assignWithinNoveltyGainColumn, axis=1)
public_transit_and_roads_bs_df['between_novelty_loss'] = public_transit_and_roads_bs_df.apply(assignBetweenNoveltyLossColumn, axis=1)
public_transit_and_roads_bs_df['between_novelty_gain'] = public_transit_and_roads_bs_df.apply(assignBetweenNoveltyGainColumn, axis=1)

public_transit_and_roads_wr_df['within_novelty_loss'] = public_transit_and_roads_wr_df.apply(assignWithinNoveltyLossColumn, axis=1)
public_transit_and_roads_wr_df['within_novelty_gain'] = public_transit_and_roads_wr_df.apply(assignWithinNoveltyGainColumn, axis=1)
public_transit_and_roads_wr_df['between_novelty_loss'] = public_transit_and_roads_wr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
public_transit_and_roads_wr_df['between_novelty_gain'] = public_transit_and_roads_wr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

public_transit_and_roads_cr_df['within_novelty_loss'] = public_transit_and_roads_cr_df.apply(assignWithinNoveltyLossColumn, axis=1)
public_transit_and_roads_cr_df['within_novelty_gain'] = public_transit_and_roads_cr_df.apply(assignWithinNoveltyGainColumn, axis=1)
public_transit_and_roads_cr_df['between_novelty_loss'] = public_transit_and_roads_cr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
public_transit_and_roads_cr_df['between_novelty_gain'] = public_transit_and_roads_cr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

public_transit_and_roads_pr_df['within_novelty_loss'] = public_transit_and_roads_pr_df.apply(assignWithinNoveltyLossColumn, axis=1)
public_transit_and_roads_pr_df['within_novelty_gain'] = public_transit_and_roads_pr_df.apply(assignWithinNoveltyGainColumn, axis=1)
public_transit_and_roads_pr_df['between_novelty_loss'] = public_transit_and_roads_pr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
public_transit_and_roads_pr_df['between_novelty_gain'] = public_transit_and_roads_pr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

within_novelty_cost_avg_map['public_transit_and_roads'] = round(public_transit_and_roads_bs_df['within_novelty'].mean(), 2)
between_novelty_cost_avg_map['public_transit_and_roads'] = round(public_transit_and_roads_bs_df['between_novelty'].mean(), 2)
within_novelty_count_avg_map['public_transit_and_roads'] = round(public_transit_and_roads_wr_df['within_novelty'].mean(), 2)
between_novelty_count_avg_map['public_transit_and_roads'] = round(public_transit_and_roads_wr_df['between_novelty'].mean(), 2)

# ## for urban greenery
urban_greenery_bs_df['within_novelty_loss'] = urban_greenery_bs_df.apply(assignWithinNoveltyLossColumn, axis=1)
urban_greenery_bs_df['within_novelty_gain'] = urban_greenery_bs_df.apply(assignWithinNoveltyGainColumn, axis=1)
urban_greenery_bs_df['between_novelty_loss'] = urban_greenery_bs_df.apply(assignBetweenNoveltyLossColumn, axis=1)
urban_greenery_bs_df['between_novelty_gain'] = urban_greenery_bs_df.apply(assignBetweenNoveltyGainColumn, axis=1)

urban_greenery_wr_df['within_novelty_loss'] = urban_greenery_wr_df.apply(assignWithinNoveltyLossColumn, axis=1)
urban_greenery_wr_df['within_novelty_gain'] = urban_greenery_wr_df.apply(assignWithinNoveltyGainColumn, axis=1)
urban_greenery_wr_df['between_novelty_loss'] = urban_greenery_wr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
urban_greenery_wr_df['between_novelty_gain'] = urban_greenery_wr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

urban_greenery_cr_df['within_novelty_loss'] = urban_greenery_cr_df.apply(assignWithinNoveltyLossColumn, axis=1)
urban_greenery_cr_df['within_novelty_gain'] = urban_greenery_cr_df.apply(assignWithinNoveltyGainColumn, axis=1)
urban_greenery_cr_df['between_novelty_loss'] = urban_greenery_cr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
urban_greenery_cr_df['between_novelty_gain'] = urban_greenery_cr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

urban_greenery_pr_df['within_novelty_loss'] = urban_greenery_pr_df.apply(assignWithinNoveltyLossColumn, axis=1)
urban_greenery_pr_df['within_novelty_gain'] = urban_greenery_pr_df.apply(assignWithinNoveltyGainColumn, axis=1)
urban_greenery_pr_df['between_novelty_loss'] = urban_greenery_pr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
urban_greenery_pr_df['between_novelty_gain'] = urban_greenery_pr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

within_novelty_cost_avg_map['urban_greenery'] = round(urban_greenery_bs_df['within_novelty'].mean(), 2)
between_novelty_cost_avg_map['urban_greenery'] = round(urban_greenery_bs_df['between_novelty'].mean(), 2)
within_novelty_count_avg_map['urban_greenery'] = round(urban_greenery_wr_df['within_novelty'].mean(), 2)
between_novelty_count_avg_map['urban_greenery'] = round(urban_greenery_wr_df['between_novelty'].mean(), 2)

# ## for sport
sport_bs_df['within_novelty_loss'] = sport_bs_df.apply(assignWithinNoveltyLossColumn, axis=1)
sport_bs_df['within_novelty_gain'] = sport_bs_df.apply(assignWithinNoveltyGainColumn, axis=1)
sport_bs_df['between_novelty_loss'] = sport_bs_df.apply(assignBetweenNoveltyLossColumn, axis=1)
sport_bs_df['between_novelty_gain'] = sport_bs_df.apply(assignBetweenNoveltyGainColumn, axis=1)

sport_wr_df['within_novelty_loss'] = sport_wr_df.apply(assignWithinNoveltyLossColumn, axis=1)
sport_wr_df['within_novelty_gain'] = sport_wr_df.apply(assignWithinNoveltyGainColumn, axis=1)
sport_wr_df['between_novelty_loss'] = sport_wr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
sport_wr_df['between_novelty_gain'] = sport_wr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

sport_cr_df['within_novelty_loss'] = sport_cr_df.apply(assignWithinNoveltyLossColumn, axis=1)
sport_cr_df['within_novelty_gain'] = sport_cr_df.apply(assignWithinNoveltyGainColumn, axis=1)
sport_cr_df['between_novelty_loss'] = sport_cr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
sport_cr_df['between_novelty_gain'] = sport_cr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

sport_pr_df['within_novelty_loss'] = sport_pr_df.apply(assignWithinNoveltyLossColumn, axis=1)
sport_pr_df['within_novelty_gain'] = sport_pr_df.apply(assignWithinNoveltyGainColumn, axis=1)
sport_pr_df['between_novelty_loss'] = sport_pr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
sport_pr_df['between_novelty_gain'] = sport_pr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

within_novelty_cost_avg_map['sport'] = round(sport_bs_df['within_novelty'].mean(), 2)
between_novelty_cost_avg_map['sport'] = round(sport_bs_df['between_novelty'].mean(), 2)
within_novelty_count_avg_map['sport'] = round(sport_wr_df['within_novelty'].mean(), 2)
between_novelty_count_avg_map['sport'] = round(sport_wr_df['between_novelty'].mean(), 2)

# ## for health
health_bs_df['within_novelty_loss'] = health_bs_df.apply(assignWithinNoveltyLossColumn, axis=1)
health_bs_df['within_novelty_gain'] = health_bs_df.apply(assignWithinNoveltyGainColumn, axis=1)
health_bs_df['between_novelty_loss'] = health_bs_df.apply(assignBetweenNoveltyLossColumn, axis=1)
health_bs_df['between_novelty_gain'] = health_bs_df.apply(assignBetweenNoveltyGainColumn, axis=1)

health_wr_df['within_novelty_loss'] = health_wr_df.apply(assignWithinNoveltyLossColumn, axis=1)
health_wr_df['within_novelty_gain'] = health_wr_df.apply(assignWithinNoveltyGainColumn, axis=1)
health_wr_df['between_novelty_loss'] = health_wr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
health_wr_df['between_novelty_gain'] = health_wr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

health_cr_df['within_novelty_loss'] = health_cr_df.apply(assignWithinNoveltyLossColumn, axis=1)
health_cr_df['within_novelty_gain'] = health_cr_df.apply(assignWithinNoveltyGainColumn, axis=1)
health_cr_df['between_novelty_loss'] = health_cr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
health_cr_df['between_novelty_gain'] = health_cr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

health_pr_df['within_novelty_loss'] = health_pr_df.apply(assignWithinNoveltyLossColumn, axis=1)
health_pr_df['within_novelty_gain'] = health_pr_df.apply(assignWithinNoveltyGainColumn, axis=1)
health_pr_df['between_novelty_loss'] = health_pr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
health_pr_df['between_novelty_gain'] = health_pr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

within_novelty_cost_avg_map['health'] = round(health_bs_df['within_novelty'].mean(), 2)
between_novelty_cost_avg_map['health'] = round(health_bs_df['between_novelty'].mean(), 2)
within_novelty_count_avg_map['health'] = round(health_wr_df['within_novelty'].mean(), 2)
between_novelty_count_avg_map['health'] = round(health_wr_df['between_novelty'].mean(), 2)

# ## for environmental protection
env_protection_bs_df['within_novelty_loss'] = env_protection_bs_df.apply(assignWithinNoveltyLossColumn, axis=1)
env_protection_bs_df['within_novelty_gain'] = env_protection_bs_df.apply(assignWithinNoveltyGainColumn, axis=1)
env_protection_bs_df['between_novelty_loss'] = env_protection_bs_df.apply(assignBetweenNoveltyLossColumn, axis=1)
env_protection_bs_df['between_novelty_gain'] = env_protection_bs_df.apply(assignBetweenNoveltyGainColumn, axis=1)

env_protection_wr_df['within_novelty_loss'] = env_protection_wr_df.apply(assignWithinNoveltyLossColumn, axis=1)
env_protection_wr_df['within_novelty_gain'] = env_protection_wr_df.apply(assignWithinNoveltyGainColumn, axis=1)
env_protection_wr_df['between_novelty_loss'] = env_protection_wr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
env_protection_wr_df['between_novelty_gain'] = env_protection_wr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

env_protection_cr_df['within_novelty_loss'] = env_protection_cr_df.apply(assignWithinNoveltyLossColumn, axis=1)
env_protection_cr_df['within_novelty_gain'] = env_protection_cr_df.apply(assignWithinNoveltyGainColumn, axis=1)
env_protection_cr_df['between_novelty_loss'] = env_protection_cr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
env_protection_cr_df['between_novelty_gain'] = env_protection_cr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

env_protection_pr_df['within_novelty_loss'] = env_protection_pr_df.apply(assignWithinNoveltyLossColumn, axis=1)
env_protection_pr_df['within_novelty_gain'] = env_protection_pr_df.apply(assignWithinNoveltyGainColumn, axis=1)
env_protection_pr_df['between_novelty_loss'] = env_protection_pr_df.apply(assignBetweenNoveltyLossColumn, axis=1)
env_protection_pr_df['between_novelty_gain'] = env_protection_pr_df.apply(assignBetweenNoveltyGainColumn, axis=1)

within_novelty_cost_avg_map['env_protection'] = round(env_protection_bs_df['within_novelty'].mean(), 2)
between_novelty_cost_avg_map['env_protection'] = round(env_protection_bs_df['between_novelty'].mean(), 2)
within_novelty_count_avg_map['env_protection'] = round(env_protection_wr_df['within_novelty'].mean(), 2)
between_novelty_count_avg_map['env_protection'] = round(env_protection_wr_df['between_novelty'].mean(), 2)


## Preparing the dataset for the boxplots

In [ ]:
# preparing budget share melted data
budget_share_data = pd.DataFrame()

bs_education = education_bs_df[['category', 'mes_bs', 'ug_bs', 'excl_mes_bs', 'excl_ug_bs']]
bs_culture = culture_bs_df[['category', 'mes_bs', 'ug_bs', 'excl_mes_bs', 'excl_ug_bs']]
bs_welfare = welfare_bs_df[['category', 'mes_bs', 'ug_bs', 'excl_mes_bs', 'excl_ug_bs']]
bs_public_space = public_space_bs_df[['category', 'mes_bs', 'ug_bs', 'excl_mes_bs', 'excl_ug_bs']]
bs_public_transit = public_transit_and_roads_bs_df[['category', 'mes_bs', 'ug_bs', 'excl_mes_bs', 'excl_ug_bs']]
bs_urban_greenery = urban_greenery_bs_df[['category', 'mes_bs', 'ug_bs', 'excl_mes_bs', 'excl_ug_bs']]
bs_sport = sport_bs_df[['category', 'mes_bs', 'ug_bs', 'excl_mes_bs', 'excl_ug_bs']]
bs_health = health_bs_df[['category', 'mes_bs', 'ug_bs', 'excl_mes_bs', 'excl_ug_bs']]
bs_env_protection = env_protection_bs_df[['category', 'mes_bs', 'ug_bs', 'excl_mes_bs', 'excl_ug_bs']]

budget_share_data = pd.concat([budget_share_data, bs_education], ignore_index=True)
budget_share_data = pd.concat([budget_share_data, bs_culture], ignore_index=True)
budget_share_data = pd.concat([budget_share_data, bs_welfare], ignore_index=True)
budget_share_data = pd.concat([budget_share_data, bs_public_space], ignore_index=True)
budget_share_data = pd.concat([budget_share_data, bs_public_transit], ignore_index=True)
budget_share_data = pd.concat([budget_share_data, bs_urban_greenery], ignore_index=True)
budget_share_data = pd.concat([budget_share_data, bs_sport], ignore_index=True)
budget_share_data = pd.concat([budget_share_data, bs_health], ignore_index=True)
budget_share_data = pd.concat([budget_share_data, bs_env_protection], ignore_index=True)

# I renamed it here only because I want to use the label in the superplot with other metrics; 
# not necessary to rename in others
budget_share_data.rename(columns={
    'ug_bs': 'UG',
    'mes_bs': 'ES',
    'excl_ug_bs': 'UG [Exclusive]',
    'excl_mes_bs': 'ES [Exclusive]'
}, inplace=True)

melted_budget_share_data = pd.melt(budget_share_data, id_vars=['category'], var_name='Winning Type', value_name='Budget Share')
melted_budget_share_data.shape


In [ ]:
# preparing winning rate melted data 
winning_rate_data = pd.DataFrame()

wr_education = education_wr_df[['category', 'mes_wr', 'ug_wr', 'excl_mes_wr', 'excl_ug_wr']]
wr_culture = culture_wr_df[['category', 'mes_wr', 'ug_wr', 'excl_mes_wr', 'excl_ug_wr']]
wr_welfare = welfare_wr_df[['category', 'mes_wr', 'ug_wr', 'excl_mes_wr', 'excl_ug_wr']]
wr_public_space = public_space_wr_df[['category', 'mes_wr', 'ug_wr', 'excl_mes_wr', 'excl_ug_wr']]
wr_public_transit = public_transit_and_roads_wr_df[['category', 'mes_wr', 'ug_wr', 'excl_mes_wr', 'excl_ug_wr']]
wr_urban_greenery = urban_greenery_wr_df[['category', 'mes_wr', 'ug_wr', 'excl_mes_wr', 'excl_ug_wr']]
wr_sport = sport_wr_df[['category', 'mes_wr', 'ug_wr', 'excl_mes_wr', 'excl_ug_wr']]
wr_health = health_wr_df[['category', 'mes_wr', 'ug_wr', 'excl_mes_wr', 'excl_ug_wr']]
wr_env_protection = env_protection_wr_df[['category', 'mes_wr', 'ug_wr', 'excl_mes_wr', 'excl_ug_wr']]

winning_rate_data = pd.concat([winning_rate_data, wr_education], ignore_index=True)
winning_rate_data = pd.concat([winning_rate_data, wr_culture], ignore_index=True)
winning_rate_data = pd.concat([winning_rate_data, wr_welfare], ignore_index=True)
winning_rate_data = pd.concat([winning_rate_data, wr_public_space], ignore_index=True)
winning_rate_data = pd.concat([winning_rate_data, wr_public_transit], ignore_index=True)
winning_rate_data = pd.concat([winning_rate_data, wr_urban_greenery], ignore_index=True)
winning_rate_data = pd.concat([winning_rate_data, wr_sport], ignore_index=True)
winning_rate_data = pd.concat([winning_rate_data, wr_health], ignore_index=True)
winning_rate_data = pd.concat([winning_rate_data, wr_env_protection], ignore_index=True)

melted_winning_rate_data = pd.melt(winning_rate_data, id_vars=['category'], var_name='Winning Type', value_name='Winning Rate')
melted_winning_rate_data.shape

In [ ]:
# preparing cost representation melted data
cost_representation_data = pd.DataFrame()

cr_education = education_cr_df[['category', 'mes_cr', 'ug_cr', 'excl_mes_cr', 'excl_ug_cr']]
cr_culture = culture_cr_df[['category', 'mes_cr', 'ug_cr', 'excl_mes_cr', 'excl_ug_cr']]
cr_welfare = welfare_cr_df[['category', 'mes_cr', 'ug_cr', 'excl_mes_cr', 'excl_ug_cr']]
cr_public_space = public_space_cr_df[['category', 'mes_cr', 'ug_cr', 'excl_mes_cr', 'excl_ug_cr']]
cr_public_transit = public_transit_and_roads_cr_df[['category', 'mes_cr', 'ug_cr', 'excl_mes_cr', 'excl_ug_cr']]
cr_urban_greenery = urban_greenery_cr_df[['category', 'mes_cr', 'ug_cr', 'excl_mes_cr', 'excl_ug_cr']]
cr_sport = sport_cr_df[['category', 'mes_cr', 'ug_cr', 'excl_mes_cr', 'excl_ug_cr']]
cr_health = health_cr_df[['category', 'mes_cr', 'ug_cr', 'excl_mes_cr', 'excl_ug_cr']]
cr_env_protection = env_protection_cr_df[['category', 'mes_cr', 'ug_cr', 'excl_mes_cr', 'excl_ug_cr']]

cost_representation_data = pd.concat([cost_representation_data, cr_education], ignore_index=True)
cost_representation_data = pd.concat([cost_representation_data, cr_culture], ignore_index=True)
cost_representation_data = pd.concat([cost_representation_data, cr_welfare], ignore_index=True)
cost_representation_data = pd.concat([cost_representation_data, cr_public_space], ignore_index=True)
cost_representation_data = pd.concat([cost_representation_data, cr_public_transit], ignore_index=True)
cost_representation_data = pd.concat([cost_representation_data, cr_urban_greenery], ignore_index=True)
cost_representation_data = pd.concat([cost_representation_data, cr_sport], ignore_index=True)
cost_representation_data = pd.concat([cost_representation_data, cr_health], ignore_index=True)
cost_representation_data = pd.concat([cost_representation_data, cr_env_protection], ignore_index=True)

melted_cost_representation_data = pd.melt(cost_representation_data, id_vars=['category'], var_name='Winning Type', value_name='Cost Representation')
melted_cost_representation_data.shape

In [ ]:
# preparing project representation melted data
project_representation_data = pd.DataFrame()

pr_education = education_pr_df[['category', 'mes_pr', 'ug_pr', 'excl_mes_pr', 'excl_ug_pr']]
pr_culture = culture_pr_df[['category', 'mes_pr', 'ug_pr', 'excl_mes_pr', 'excl_ug_pr']]
pr_welfare = welfare_pr_df[['category', 'mes_pr', 'ug_pr', 'excl_mes_pr', 'excl_ug_pr']]
pr_public_space = public_space_pr_df[['category', 'mes_pr', 'ug_pr', 'excl_mes_pr', 'excl_ug_pr']]
pr_public_transit = public_transit_and_roads_pr_df[['category', 'mes_pr', 'ug_pr', 'excl_mes_pr', 'excl_ug_pr']]
pr_urban_greenery = urban_greenery_pr_df[['category', 'mes_pr', 'ug_pr', 'excl_mes_pr', 'excl_ug_pr']]
pr_sport = sport_pr_df[['category', 'mes_pr', 'ug_pr', 'excl_mes_pr', 'excl_ug_pr']]
pr_health = health_pr_df[['category', 'mes_pr', 'ug_pr', 'excl_mes_pr', 'excl_ug_pr']]
pr_env_protection = env_protection_pr_df[['category', 'mes_pr', 'ug_pr', 'excl_mes_pr', 'excl_ug_pr']]

project_representation_data = pd.concat([project_representation_data, pr_education], ignore_index=True)
project_representation_data = pd.concat([project_representation_data, pr_culture], ignore_index=True)
project_representation_data = pd.concat([project_representation_data, pr_welfare], ignore_index=True)
project_representation_data = pd.concat([project_representation_data, pr_public_space], ignore_index=True)
project_representation_data = pd.concat([project_representation_data, pr_public_transit], ignore_index=True)
project_representation_data = pd.concat([project_representation_data, pr_urban_greenery], ignore_index=True)
project_representation_data = pd.concat([project_representation_data, pr_sport], ignore_index=True)
project_representation_data = pd.concat([project_representation_data, pr_health], ignore_index=True)
project_representation_data = pd.concat([project_representation_data, pr_env_protection], ignore_index=True)

melted_project_representation_data = pd.melt(project_representation_data, id_vars=['category'], var_name='Winning Type', value_name='Project Representation')
melted_project_representation_data.shape

In [ ]:
# for within and between novelties (loss and gain) in terms of winning costs
# preparing cost novelty melted data
cost_novelty_data = pd.DataFrame()

cost_novelty_education = education_bs_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
cost_novelty_culture = culture_bs_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
cost_novelty_welfare = welfare_bs_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
cost_novelty_public_space = public_space_bs_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
cost_novelty_public_transit = public_transit_and_roads_bs_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
cost_novelty_urban_greenery = urban_greenery_bs_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
cost_novelty_sport = sport_bs_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
cost_novelty_health = health_bs_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
cost_novelty_env_protection = env_protection_bs_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]

cost_novelty_data = pd.concat([cost_novelty_data, cost_novelty_education], ignore_index=True)
cost_novelty_data = pd.concat([cost_novelty_data, cost_novelty_culture], ignore_index=True)
cost_novelty_data = pd.concat([cost_novelty_data, cost_novelty_welfare], ignore_index=True)
cost_novelty_data = pd.concat([cost_novelty_data, cost_novelty_public_space], ignore_index=True)
cost_novelty_data = pd.concat([cost_novelty_data, cost_novelty_public_transit], ignore_index=True)
cost_novelty_data = pd.concat([cost_novelty_data, cost_novelty_urban_greenery], ignore_index=True)
cost_novelty_data = pd.concat([cost_novelty_data, cost_novelty_sport], ignore_index=True)
cost_novelty_data = pd.concat([cost_novelty_data, cost_novelty_health], ignore_index=True)
cost_novelty_data = pd.concat([cost_novelty_data, cost_novelty_env_protection], ignore_index=True)

# I renamed it here only because I want to use the label in the superplot with other metrics; not necessary to rename in others
cost_novelty_data.rename(columns={
    'within_novelty_loss': 'Within Novelty Loss',
    'within_novelty_gain': 'Within Novelty Gain',
    'between_novelty_loss': 'Between Novelty Loss',
    'between_novelty_gain': 'Between Novelty Gain'
}, inplace=True)

melted_cost_novelty_data = pd.melt(cost_novelty_data, id_vars=['category'], var_name='Winning Type', value_name='Winning Costs')
melted_cost_novelty_data.shape


In [ ]:
# for within and between novelties (loss and gain) in terms of winning projects
# preparing count novelty melted data 
count_novelty_data = pd.DataFrame()

count_novelty_education = education_wr_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
count_novelty_culture = culture_wr_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
count_novelty_welfare = welfare_wr_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
count_novelty_public_space = public_space_wr_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
count_novelty_public_transit = public_transit_and_roads_wr_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
count_novelty_urban_greenery = urban_greenery_wr_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
count_novelty_sport = sport_wr_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
count_novelty_health = health_wr_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]
count_novelty_env_protection = env_protection_wr_df[['category', 'within_novelty_loss', 'within_novelty_gain', 'between_novelty_loss', 'between_novelty_gain']]

count_novelty_data = pd.concat([count_novelty_data, count_novelty_education], ignore_index=True)
count_novelty_data = pd.concat([count_novelty_data, count_novelty_culture], ignore_index=True)
count_novelty_data = pd.concat([count_novelty_data, count_novelty_welfare], ignore_index=True)
count_novelty_data = pd.concat([count_novelty_data, count_novelty_public_space], ignore_index=True)
count_novelty_data = pd.concat([count_novelty_data, count_novelty_public_transit], ignore_index=True)
count_novelty_data = pd.concat([count_novelty_data, count_novelty_urban_greenery], ignore_index=True)
count_novelty_data = pd.concat([count_novelty_data, count_novelty_sport], ignore_index=True)
count_novelty_data = pd.concat([count_novelty_data, count_novelty_health], ignore_index=True)
count_novelty_data = pd.concat([count_novelty_data, count_novelty_env_protection], ignore_index=True)

melted_count_novelty_data = pd.melt(count_novelty_data, id_vars=['category'], var_name='Winning Type', value_name='Winning Projects')
melted_count_novelty_data.shape

### Box plot for all 4 metrics across 9 categories

In [ ]:
fig, axes = plt.subplots(1, 6, figsize=(24, 30), sharex=True, sharey=True)

impact_color_palette = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
novelty_color_palette = ['#a6d854','#ffd92f','#e5c494','#b3b3b3']

sns.boxplot(y='category', ax=axes[0], x='Budget Share', hue='Winning Type', data=melted_budget_share_data, showmeans=True, medianprops={'visible': False}, flierprops={"marker": "x"}, width=0.7, meanprops={"marker": '*', "markerfacecolor":"black", "markeredgecolor":"black"}, palette=impact_color_palette, dodge=True, gap=0.2)
sns.boxplot(y='category', ax=axes[1], x='Winning Rate', hue='Winning Type', data=melted_winning_rate_data, showmeans=True, medianprops={'visible': False}, flierprops={"marker": "x"}, width=0.7, meanprops={"marker":"*", "markerfacecolor":"black", "markeredgecolor":"black"}, palette=impact_color_palette, dodge=True, gap=0.2)
sns.boxplot(y='category', ax=axes[2], x='Cost Representation', hue='Winning Type', data=melted_cost_representation_data, showmeans=True, medianprops={'visible': False}, flierprops={"marker": "x"}, width=0.7, meanprops={"marker":"*", "markerfacecolor":"black", "markeredgecolor":"black"}, palette=impact_color_palette, dodge=True, gap=0.2)
sns.boxplot(y='category', ax=axes[3], x='Project Representation', hue='Winning Type', data=melted_project_representation_data, showmeans=True, medianprops={'visible': False}, flierprops={"marker": "x"}, width=0.7, meanprops={"marker":"*", "markerfacecolor":"black", "markeredgecolor":"black"}, palette=impact_color_palette, dodge=True, gap=0.2)
sns.boxplot(y='category', ax=axes[4], x='Winning Costs', hue='Winning Type', data=melted_cost_novelty_data, showmeans=True, medianprops={'visible': False}, flierprops={"marker": "x"}, width=0.7, meanprops={"marker":"*", "markerfacecolor":"black", "markeredgecolor":"black"}, palette=novelty_color_palette, dodge=True, gap=0.2)
sns.boxplot(y='category', ax=axes[5], x='Winning Projects', hue='Winning Type', data=melted_count_novelty_data, showmeans=True, medianprops={'visible': False}, flierprops={"marker": "x"}, width=0.7, meanprops={"marker":"*", "markerfacecolor":"black", "markeredgecolor":"black"}, palette=novelty_color_palette, dodge=True, gap=0.2)



# Customize legend
handles1, labels1 = axes[0].get_legend_handles_labels()
handles2, labels2 = axes[4].get_legend_handles_labels()

final_handles = []
final_labels = []

for h1 in handles1:
    final_handles.append(h1)
for h2 in handles2:
    final_handles.append(h2)

for l1 in labels1:
    final_labels.append(l1)
for l2 in labels2:
    final_labels.append(l2)

# Create legend outside of the main plot
fig.legend(final_handles, final_labels, loc='lower center', bbox_to_anchor=(0.55, 1.005), ncols=8, frameon=False, fontsize=16)
fig.subplots_adjust(right=1)  # Adjust right side to fit legend

# Text overlay for overall novelty average of in between and within novelties
for idx, ((within_cost_key, within_cost_value), (between_cost_key, between_cost_value)) in enumerate(zip(within_novelty_cost_avg_map.items(), between_novelty_cost_avg_map.items())):
    if(within_cost_key == between_cost_key):
        axes[4].text(0, idx - 0.35, f'Within Novelty:{within_cost_value}\nBetween Novelty:{between_cost_value}', fontdict={'size':16})

for idx, ((within_count_key, within_count_value), (between_count_key, between_count_value)) in enumerate(zip(within_novelty_count_avg_map.items(), between_novelty_count_avg_map.items())):
    if(within_count_key == between_count_key):
        axes[5].text(0, idx - 0.35, f'Within Novelty:{within_count_value}\nBetween Novelty:{between_count_value}', fontdict={'size':16})

axes[0].legend().set_visible(False)
axes[1].legend().set_visible(False)
axes[2].legend().set_visible(False)
axes[3].legend().set_visible(False)
axes[4].legend().set_visible(False)
axes[5].legend().set_visible(False)

plt.xticks(np.arange(0, 1.1, 0.25), fontsize=12)

axes[0].set_ylabel('')
axes[1].set_ylabel('')
axes[2].set_ylabel('')
axes[3].set_ylabel('')
axes[4].set_ylabel('')
axes[5].set_ylabel('')

axes[0].set_xlabel('')
axes[1].set_xlabel('')
axes[2].set_xlabel('')
axes[3].set_xlabel('')
axes[4].set_xlabel('')
axes[5].set_xlabel('')

axes[0].set_title('Budget Share', fontsize=18, fontdict={'fontweight': 'bold'})
axes[1].set_title('Winning Rate', fontsize=18, fontdict={'fontweight': 'bold'})
axes[2].set_title('Cost Representation', fontsize=18, fontdict={'fontweight': 'bold'})
axes[3].set_title('Project Representation', fontsize=18, fontdict={'fontweight': 'bold'})
axes[4].set_title('Winning Costs', fontsize=18, fontdict={'fontweight': 'bold'})
axes[5].set_title('Winning Projects', fontsize=18, fontdict={'fontweight': 'bold'})

axes[0].tick_params(axis='both', labelsize=16, width=4, length=8, pad=8)
axes[1].tick_params(axis='both', labelsize=16, width=4, length=8, pad=8)
axes[2].tick_params(axis='both', labelsize=16, width=4, length=8, pad=8)
axes[3].tick_params(axis='both', labelsize=16, width=4, length=8, pad=8)
axes[4].tick_params(axis='both', labelsize=16, width=4, length=8, pad=8)
axes[5].tick_params(axis='both', labelsize=16, width=4, length=8, pad=8)

axes[0].set_xticklabels(axes[0].get_xticklabels(), fontweight='bold') 
axes[1].set_xticklabels(axes[1].get_xticklabels(), fontweight='bold') 
axes[2].set_xticklabels(axes[2].get_xticklabels(), fontweight='bold') 
axes[3].set_xticklabels(axes[3].get_xticklabels(), fontweight='bold') 
axes[4].set_xticklabels(axes[4].get_xticklabels(), fontweight='bold') 
axes[5].set_xticklabels(axes[5].get_xticklabels(), fontweight='bold') 

axes[0].grid(axis='both', which='major', color='gray', alpha=0.3)
axes[1].grid(axis='both', which='major', color='gray', alpha=0.3)
axes[2].grid(axis='both', which='major', color='gray', alpha=0.3)
axes[3].grid(axis='both', which='major', color='gray', alpha=0.3)
axes[4].grid(axis='both', which='major', color='gray', alpha=0.3)
axes[5].grid(axis='both', which='major', color='gray', alpha=0.3)

# # Get the y-labels
labels = axes[0].get_yticklabels()


# Set bold font for y-labels
for label in labels:
    label.set_fontweight('bold')
    label.set_fontsize(18)
    label.set_rotation(90)
    label.set_verticalalignment('center') 

# Adjust the border thickness of subplots
for ax in axes:
    for spine in ax.spines.values():
        spine.set_linewidth(2)  # Set border thickness to 2 points

plt.tight_layout()
plt.show()